In [11]:
import json
reflexion_results = []
simple_acc = 0
reflexion_acc = 0
simple_err = 0
dataset_name = "torchdata"

with open(f"/ML-A100/home/gujiasheng/reflexion/programming_runs/root/test_reflexion_{dataset_name}_gpt4/real_{dataset_name}_eval_v3_api_5._reflexion_2_gpt-4_pass_at_k_1_py.jsonl", 'r') as f:
    for line in f:
        reflexion_results.append(json.loads(line))
for reflexion_result in reflexion_results:
    if reflexion_result["is_solved"] and reflexion_result["reflections"] == []:
        simple_acc += 1
    elif not reflexion_result["is_solved"] and reflexion_result["reflections"] != []:
        simple_err += 1
    elif reflexion_result["is_solved"] and reflexion_result["reflections"] != []:
        reflexion_acc += 1
    
    if  not reflexion_result["is_solved"] and reflexion_result["reflections"] != []:
        # pass
        print("*"*50)
        # print(reflexion_result["reflections"][0])
        print(reflexion_result['apis'][0][0])
        print(reflexion_result["prompt"]+'\n'+reflexion_result["canonical_solution"][0])
        print("-"*50)
        print(reflexion_result["implementations"][-1])
        print("-"*50)
        print(reflexion_result["test"])

print(simple_acc)
print(simple_err)
print(simple_acc+reflexion_acc)

**************************************************
api_path: torchdata.datapipes.iter.fork
api_doc: 
    Creates multiple instances of the same Iterable DataPipe (functional name: ``fork``).

    Args:
        datapipe: Iterable DataPipe being copied
        num_instances: number of instances of the datapipe to create
        buffer_size: this restricts how far ahead the leading child DataPipe
           can read relative to the slowest child DataPipe.
           Defaults to ``1000``. Use ``-1`` for the unlimited buffer.

    Example:
        >>> from torchdata.datapipes.iter import IterableWrapper
        >>> source_dp = IterableWrapper(range(5))
        >>> dp1, dp2 = source_dp.fork(num_instances=2)
        >>> list(dp1)
        [0, 1, 2, 3, 4]
        >>> list(dp2)
        [0, 1, 2, 3, 4]
    
api_signature: (datapipe: torch.utils.data.dataset.IterDataPipe, num_instances: int, buffer_size: int = 1000)
from torchdata.datapipes.iter import IterableWrapper

source_dp = IterableWrapper(

In [9]:
print("Your improved implementation is not complete. You have calculated the number of rows (nrows) but you haven't used it to create the matrix of sub sequences. You should use the numpy function `np.lib.stride_tricks.sliding_window_view` to create the matrix. Here is the complete implementation:\n\n```python\nimport numpy as np\n\ndef broadcasting_app(a, L, S):  # Window len = L, Stride len/stepsize = S\n    \"\"\"\n    I want to create a matrix of sub sequences from this array of length L with stride S.\n    Return the numpy array of sub sequences.\n    \"\"\"\n    nrows")

Your improved implementation is not complete. You have calculated the number of rows (nrows) but you haven't used it to create the matrix of sub sequences. You should use the numpy function `np.lib.stride_tricks.sliding_window_view` to create the matrix. Here is the complete implementation:

```python
import numpy as np

def broadcasting_app(a, L, S):  # Window len = L, Stride len/stepsize = S
    """
    I want to create a matrix of sub sequences from this array of length L with stride S.
    Return the numpy array of sub sequences.
    """
    nrows


In [7]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

def broadcasting_app(a, L, S):
    # Ensure that a is a numpy array
    a = np.asarray(a)

    # Check if the input array is large enough for the given window length
    if L > a.size:
        raise ValueError("Window length L cannot be greater than the size of the array.")

    # Create a sliding window view of the array
    window_view = sliding_window_view(a, window_shape=L)

    # Slice the window view according to the stride
    return window_view[::S]

# Example usage
a = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
L = 3  # Window length
S = 2  # Stride length
result = broadcasting_app(a, L, S)
print(result)

[[1 2 3]
 [3 4 5]
 [5 6 7]
 [7 8 9]]


In [1]:
print('# coding=utf-8\n# Copyright 2018 The Google AI Language Team Authors, Facebook AI Research authors and The HuggingFace Inc. team.\n# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#     http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\nimport collections\nimport functools\nimport gc\nimport importlib.metadata\nimport inspect\nimport json\nimport os\nimport re\nimport shutil\nimport tempfile\nimport warnings\nfrom contextlib import contextmanager\nfrom dataclasses import dataclass\nfrom functools import partial, wraps\nfrom typing import Any, Callable, Dict, List, Optional, Tuple, Union\n\nimport torch\nfrom packaging import version\nfrom torch import Tensor, nn\nfrom torch.nn import CrossEntropyLoss, Identity\nfrom torch.utils.checkpoint import checkpoint\n\nfrom .activations import get_activation\nfrom .configuration_utils import PretrainedConfig\nfrom .dynamic_module_utils import custom_object_save\nfrom .generation import GenerationConfig, GenerationMixin\nfrom .integrations import PeftAdapterMixin, deepspeed_config, is_deepspeed_zero3_enabled\nfrom .pytorch_utils import (  # noqa: F401\n    Conv1D,\n    apply_chunking_to_forward,\n    find_pruneable_heads_and_indices,\n    id_tensor_storage,\n    prune_conv1d_layer,\n    prune_layer,\n    prune_linear_layer,\n)\nfrom .utils import (\n    ADAPTER_SAFE_WEIGHTS_NAME,\n    ADAPTER_WEIGHTS_NAME,\n    CONFIG_NAME,\n    DUMMY_INPUTS,\n    FLAX_WEIGHTS_NAME,\n    SAFE_WEIGHTS_INDEX_NAME,\n    SAFE_WEIGHTS_NAME,\n    TF2_WEIGHTS_NAME,\n    TF_WEIGHTS_NAME,\n    WEIGHTS_INDEX_NAME,\n    WEIGHTS_NAME,\n    ContextManagers,\n    ModelOutput,\n    PushToHubMixin,\n    cached_file,\n    copy_func,\n    download_url,\n    extract_commit_hash,\n    has_file,\n    is_accelerate_available,\n    is_auto_awq_available,\n    is_auto_gptq_available,\n    is_bitsandbytes_available,\n    is_flash_attn_2_available,\n    is_offline_mode,\n    is_optimum_available,\n    is_peft_available,\n    is_remote_url,\n    is_safetensors_available,\n    is_torch_tpu_available,\n    logging,\n    replace_return_docstrings,\n    strtobool,\n)\nfrom .utils.hub import convert_file_size_to_int, get_checkpoint_shard_files\nfrom .utils.import_utils import (\n    ENV_VARS_TRUE_VALUES,\n    is_sagemaker_mp_enabled,\n    is_torch_fx_proxy,\n    is_torchdynamo_compiling,\n)\nfrom .utils.quantization_config import AwqConfig, BitsAndBytesConfig, GPTQConfig, QuantizationMethod\nfrom .utils.versions import require_version_core\n\n\nXLA_USE_BF16 = os.environ.get("XLA_USE_BF16", "0").upper()\nXLA_DOWNCAST_BF16 = os.environ.get("XLA_DOWNCAST_BF16", "0").upper()\n\nif is_accelerate_available():\n    from accelerate import dispatch_model, infer_auto_device_map, init_empty_weights\n    from accelerate.hooks import add_hook_to_module\n    from accelerate.utils import (\n        check_tied_parameters_on_same_device,\n        find_tied_parameters,\n        get_balanced_memory,\n        get_max_memory,\n        load_offloaded_weights,\n        offload_weight,\n        save_offload_index,\n        set_module_tensor_to_device,\n    )\n\nif is_safetensors_available():\n    from safetensors import safe_open\n    from safetensors.torch import load_file as safe_load_file\n    from safetensors.torch import save_file as safe_save_file\n\nlogger = logging.get_logger(__name__)\n\n\n_init_weights = True\n\n\ndef is_fsdp_enabled():\n    return (\n        torch.distributed.is_available()\n        and torch.distributed.is_initialized()\n        and strtobool(os.environ.get("ACCELERATE_USE_FSDP", "False")) == 1\n        and strtobool(os.environ.get("FSDP_CPU_RAM_EFFICIENT_LOADING", "False")) == 1\n    )\n\n\ndef is_fsdp_enabled_and_dist_rank_0():\n    return is_fsdp_enabled() and int(os.environ.get("LOCAL_RANK", -1)) == 0\n\n\nif is_sagemaker_mp_enabled():\n    import smdistributed.modelparallel.torch as smp\n    from smdistributed.modelparallel import __version__ as SMP_VERSION\n\n    IS_SAGEMAKER_MP_POST_1_10 = version.parse(SMP_VERSION) >= version.parse("1.10")\nelse:\n    IS_SAGEMAKER_MP_POST_1_10 = False\n\nif is_peft_available():\n    from .utils import find_adapter_config_file\n\n\n@contextmanager\ndef no_init_weights(_enable=True):\n    """\n    Context manager to globally disable weight initialization to speed up loading large models.\n\n    TODO(Patrick): Delete safety argument `_enable=True` at next major version. .\n    """\n    global _init_weights\n    old_init_weights = _init_weights\n    if _enable:\n        _init_weights = False\n    try:\n        yield\n    finally:\n        _init_weights = old_init_weights\n\n\ndef get_parameter_device(parameter: Union[nn.Module, GenerationMixin, "ModuleUtilsMixin"]):\n    try:\n        return next(parameter.parameters()).device\n    except StopIteration:\n        # For nn.DataParallel compatibility in PyTorch 1.5\n\n        def find_tensor_attributes(module: nn.Module) -> List[Tuple[str, Tensor]]:\n            tuples = [(k, v) for k, v in module.__dict__.items() if torch.is_tensor(v)]\n            return tuples\n\n        gen = parameter._named_members(get_members_fn=find_tensor_attributes)\n        first_tuple = next(gen)\n        return first_tuple[1].device\n\n\ndef get_first_parameter_dtype(parameter: Union[nn.Module, GenerationMixin, "ModuleUtilsMixin"]):\n    """\n    Returns the first parameter dtype (can be non-floating) or asserts if none were found.\n    """\n    try:\n        return next(parameter.parameters()).dtype\n    except StopIteration:\n        # For nn.DataParallel compatibility in PyTorch > 1.5\n\n        def find_tensor_attributes(module: nn.Module) -> List[Tuple[str, Tensor]]:\n            tuples = [(k, v) for k, v in module.__dict__.items() if torch.is_tensor(v)]\n            return tuples\n\n        gen = parameter._named_members(get_members_fn=find_tensor_attributes)\n        first_tuple = next(gen)\n        return first_tuple[1].dtype\n\n\ndef get_parameter_dtype(parameter: Union[nn.Module, GenerationMixin, "ModuleUtilsMixin"]):\n    """\n    Returns the first found floating dtype in parameters if there is one, otherwise returns the last dtype it found.\n    """\n    last_dtype = None\n    for t in parameter.parameters():\n        last_dtype = t.dtype\n        if t.is_floating_point():\n            # Adding fix for https://github.com/pytorch/xla/issues/4152\n            # Fixes issue where the model code passes a value that is out of range for XLA_USE_BF16=1\n            # and XLA_DOWNCAST_BF16=1 so the conversion would cast it to -inf\n            # NOTE: `is_torch_tpu_available()` is checked last as it induces a graph break in torch dynamo\n            if XLA_USE_BF16 in ENV_VARS_TRUE_VALUES and is_torch_tpu_available():\n                return torch.bfloat16\n            if XLA_DOWNCAST_BF16 in ENV_VARS_TRUE_VALUES and is_torch_tpu_available():\n                if t.dtype == torch.float:\n                    return torch.bfloat16\n                if t.dtype == torch.double:\n                    return torch.float32\n            return t.dtype\n\n    if last_dtype is not None:\n        # if no floating dtype was found return whatever the first dtype is\n        return last_dtype\n\n    # For nn.DataParallel compatibility in PyTorch > 1.5\n    def find_tensor_attributes(module: nn.Module) -> List[Tuple[str, Tensor]]:\n        tuples = [(k, v) for k, v in module.__dict__.items() if torch.is_tensor(v)]\n        return tuples\n\n    gen = parameter._named_members(get_members_fn=find_tensor_attributes)\n    last_tuple = None\n    for tuple in gen:\n        last_tuple = tuple\n        if tuple[1].is_floating_point():\n            return tuple[1].dtype\n\n    if last_tuple is not None:\n        # fallback to the last dtype\n        return last_tuple[1].dtype\n\n    # fallback to buffer dtype\n    for t in parameter.buffers():\n        last_dtype = t.dtype\n        if t.is_floating_point():\n            return t.dtype\n    return last_dtype\n\n\ndef get_state_dict_float_dtype(state_dict):\n    """\n    Returns the first found floating dtype in `state_dict` or asserts if none were found.\n    """\n    for t in state_dict.values():\n        if t.is_floating_point():\n            return t.dtype\n\n    raise ValueError("couldn\'t find any floating point dtypes in state_dict")\n\n\ndef get_state_dict_dtype(state_dict):\n    """\n    Returns the first found floating dtype in `state_dict` if there is one, otherwise returns the first dtype.\n    """\n    for t in state_dict.values():\n        if t.is_floating_point():\n            return t.dtype\n\n    # if no floating dtype was found return whatever the first dtype is\n    else:\n        return next(state_dict.values()).dtype\n\n\ndef dtype_byte_size(dtype):\n    """\n    Returns the size (in bytes) occupied by one parameter of type `dtype`.\n\n    Example:\n\n    ```py\n    >>> dtype_byte_size(torch.float32)\n    4\n    ```\n    """\n    if dtype == torch.bool:\n        return 1 / 8\n    bit_search = re.search(r"[^\\d](\\d+)$", str(dtype))\n    if bit_search is None:\n        raise ValueError(f"`dtype` is not a valid dtype: {dtype}.")\n    bit_size = int(bit_search.groups()[0])\n    return bit_size // 8\n\n\ndef shard_checkpoint(\n    state_dict: Dict[str, torch.Tensor], max_shard_size: Union[int, str] = "10GB", weights_name: str = WEIGHTS_NAME\n):\n    """\n    Splits a model state dictionary in sub-checkpoints so that the final size of each sub-checkpoint does not exceed a\n    given size.\n\n    The sub-checkpoints are determined by iterating through the `state_dict` in the order of its keys, so there is no\n    optimization made to make each sub-checkpoint as close as possible to the maximum size passed. For example, if the\n    limit is 10GB and we have weights of sizes [6GB, 6GB, 2GB, 6GB, 2GB, 2GB] they will get sharded as [6GB], [6+2GB],\n    [6+2+2GB] and not [6+2+2GB], [6+2GB], [6GB].\n\n    <Tip warning={true}>\n\n    If one of the model\'s weight is bigger than `max_shard_size`, it will end up in its own sub-checkpoint which will\n    have a size greater than `max_shard_size`.\n\n    </Tip>\n\n    Args:\n        state_dict (`Dict[str, torch.Tensor]`): The state dictionary of a model to save.\n        max_shard_size (`int` or `str`, *optional*, defaults to `"10GB"`):\n            The maximum size of each sub-checkpoint. If expressed as a string, needs to be digits followed by a unit\n            (like `"5MB"`).\n        weights_name (`str`, *optional*, defaults to `"pytorch_model.bin"`):\n            The name of the model save file.\n    """\n    max_shard_size = convert_file_size_to_int(max_shard_size)\n\n    sharded_state_dicts = [{}]\n    last_block_size = 0\n    total_size = 0\n    storage_id_to_block = {}\n\n    for key, weight in state_dict.items():\n        # when bnb serialization is used the weights in the state dict can be strings\n        # check: https://github.com/huggingface/transformers/pull/24416 for more details\n        if isinstance(weight, str):\n            continue\n        else:\n            storage_id = id_tensor_storage(weight)\n\n        # If a `weight` shares the same underlying storage as another tensor, we put `weight` in the same `block`\n        if storage_id in storage_id_to_block:\n            block_id = storage_id_to_block[storage_id]\n            sharded_state_dicts[block_id][key] = weight\n            continue\n\n        weight_size = weight.numel() * dtype_byte_size(weight.dtype)\n\n        # If this weight is going to tip up over the maximal size, we split, but only if we have put at least one\n        # weight in the current shard.\n        if last_block_size + weight_size > max_shard_size and len(sharded_state_dicts[-1]) > 0:\n            sharded_state_dicts.append({})\n            last_block_size = 0\n\n        sharded_state_dicts[-1][key] = weight\n        last_block_size += weight_size\n        total_size += weight_size\n        storage_id_to_block[storage_id] = len(sharded_state_dicts) - 1\n\n    # If we only have one shard, we return it\n    if len(sharded_state_dicts) == 1:\n        return {weights_name: sharded_state_dicts[0]}, None\n\n    # Otherwise, let\'s build the index\n    weight_map = {}\n    shards = {}\n    for idx, shard in enumerate(sharded_state_dicts):\n        shard_file = weights_name.replace(".bin", f"-{idx+1:05d}-of-{len(sharded_state_dicts):05d}.bin")\n        shard_file = shard_file.replace(\n            ".safetensors", f"-{idx + 1:05d}-of-{len(sharded_state_dicts):05d}.safetensors"\n        )\n        shards[shard_file] = shard\n        for key in shard.keys():\n            weight_map[key] = shard_file\n\n    # Add the metadata\n    metadata = {"total_size": total_size}\n    index = {"metadata": metadata, "weight_map": weight_map}\n    return shards, index\n\n\ndef load_sharded_checkpoint(model, folder, strict=True, prefer_safe=True):\n    """\n    This is the same as\n    [`torch.nn.Module.load_state_dict`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=load_state_dict#torch.nn.Module.load_state_dict)\n    but for a sharded checkpoint.\n\n    This load is performed efficiently: each checkpoint shard is loaded one by one in RAM and deleted after being\n    loaded in the model.\n\n    Args:\n        model (`torch.nn.Module`): The model in which to load the checkpoint.\n        folder (`str` or `os.PathLike`): A path to a folder containing the sharded checkpoint.\n        strict (`bool`, *optional`, defaults to `True`):\n            Whether to strictly enforce that the keys in the model state dict match the keys in the sharded checkpoint.\n        prefer_safe (`bool`, *optional*, defaults to `False`)\n            If both safetensors and PyTorch save files are present in checkpoint and `prefer_safe` is True, the\n            safetensors files will be loaded. Otherwise, PyTorch files are always loaded when possible.\n\n    Returns:\n        `NamedTuple`: A named tuple with `missing_keys` and `unexpected_keys` fields\n            - `missing_keys` is a list of str containing the missing keys\n            - `unexpected_keys` is a list of str containing the unexpected keys\n    """\n    # Load the index\n    index_file = os.path.join(folder, WEIGHTS_INDEX_NAME)\n    safe_index_file = os.path.join(folder, SAFE_WEIGHTS_INDEX_NAME)\n\n    index_present = os.path.isfile(index_file)\n    safe_index_present = os.path.isfile(safe_index_file)\n\n    if not index_present and not (safe_index_present and is_safetensors_available()):\n        filenames = (\n            (WEIGHTS_INDEX_NAME, SAFE_WEIGHTS_INDEX_NAME) if is_safetensors_available() else (WEIGHTS_INDEX_NAME,)\n        )\n        raise ValueError(f"Can\'t find a checkpoint index ({\' or \'.join(filenames)}) in {folder}.")\n\n    load_safe = False\n    if safe_index_present:\n        if prefer_safe:\n            if is_safetensors_available():\n                load_safe = True  # load safe due to preference\n            else:\n                logger.warning(\n                    f"Cannot load sharded checkpoint at {folder} safely since safetensors is not installed!"\n                )\n        elif not index_present:\n            load_safe = True  # load safe since we have no other choice\n\n    load_index = safe_index_file if load_safe else index_file\n\n    with open(load_index, "r", encoding="utf-8") as f:\n        index = json.load(f)\n\n    shard_files = list(set(index["weight_map"].values()))\n\n    # If strict=True, error before loading any of the state dicts.\n    loaded_keys = index["weight_map"].keys()\n    model_keys = model.state_dict().keys()\n    missing_keys = [key for key in model_keys if key not in loaded_keys]\n    unexpected_keys = [key for key in loaded_keys if key not in model_keys]\n    if strict and (len(missing_keys) > 0 or len(unexpected_keys) > 0):\n        error_message = f"Error(s) in loading state_dict for {model.__class__.__name__}"\n        if len(missing_keys) > 0:\n            str_missing_keys = ",".join([f\'"{k}"\' for k in missing_keys])\n            error_message += f"\\nMissing key(s): {str_missing_keys}."\n        if len(unexpected_keys) > 0:\n            str_unexpected_keys = ",".join([f\'"{k}"\' for k in unexpected_keys])\n            error_message += f"\\nMissing key(s): {str_unexpected_keys}."\n        raise RuntimeError(error_message)\n\n    loader = safe_load_file if load_safe else partial(torch.load, map_location="cpu")\n\n    for shard_file in shard_files:\n        state_dict = loader(os.path.join(folder, shard_file))\n        model.load_state_dict(state_dict, strict=False)\n\n        # Make sure memory is freed before we load the next state dict.\n        del state_dict\n        gc.collect()\n\n    # Return the same thing as PyTorch load_state_dict function.\n    return torch.nn.modules.module._IncompatibleKeys(missing_keys, unexpected_keys)\n\n\ndef load_state_dict(checkpoint_file: Union[str, os.PathLike]):\n    """\n    Reads a PyTorch checkpoint file, returning properly formatted errors if they arise.\n    """\n    if checkpoint_file.endswith(".safetensors") and is_safetensors_available():\n        # Check format of the archive\n        with safe_open(checkpoint_file, framework="pt") as f:\n            metadata = f.metadata()\n        if metadata.get("format") not in ["pt", "tf", "flax"]:\n            raise OSError(\n                f"The safetensors archive passed at {checkpoint_file} does not contain the valid metadata. Make sure "\n                "you save your model with the `save_pretrained` method."\n            )\n        return safe_load_file(checkpoint_file)\n    try:\n        if (\n            (is_deepspeed_zero3_enabled() or is_fsdp_enabled())\n            and torch.distributed.is_initialized()\n            and torch.distributed.get_rank() > 0\n        ):\n            map_location = "meta"\n        else:\n            map_location = "cpu"\n        return torch.load(checkpoint_file, map_location=map_location)\n    except Exception as e:\n        try:\n            with open(checkpoint_file) as f:\n                if f.read(7) == "version":\n                    raise OSError(\n                        "You seem to have cloned a repository without having git-lfs installed. Please install "\n                        "git-lfs and run `git lfs install` followed by `git lfs pull` in the folder "\n                        "you cloned."\n                    )\n                else:\n                    raise ValueError(\n                        f"Unable to locate the file {checkpoint_file} which is necessary to load this pretrained "\n                        "model. Make sure you have saved the model properly."\n                    ) from e\n        except (UnicodeDecodeError, ValueError):\n            raise OSError(\n                f"Unable to load weights from pytorch checkpoint file for \'{checkpoint_file}\' "\n                f"at \'{checkpoint_file}\'. "\n                "If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True."\n            )\n\n\ndef set_initialized_submodules(model, state_dict_keys):\n    """\n    Sets the `_is_hf_initialized` flag in all submodules of a given model when all its weights are in the loaded state\n    dict.\n    """\n    for module_name, module in model.named_modules():\n        loaded_keys = [k.replace(f"{module_name}.", "") for k in state_dict_keys if k.startswith(f"{module_name}.")]\n        if len(set(module.state_dict().keys()) - set(loaded_keys)) == 0:\n            module._is_hf_initialized = True\n\n\ndef _load_state_dict_into_model(model_to_load, state_dict, start_prefix):\n    # Convert old format to new format if needed from a PyTorch state_dict\n    old_keys = []\n    new_keys = []\n    for key in state_dict.keys():\n        new_key = None\n        if "gamma" in key:\n            new_key = key.replace("gamma", "weight")\n        if "beta" in key:\n            new_key = key.replace("beta", "bias")\n        if new_key:\n            old_keys.append(key)\n            new_keys.append(new_key)\n    for old_key, new_key in zip(old_keys, new_keys):\n        state_dict[new_key] = state_dict.pop(old_key)\n\n    # copy state_dict so _load_from_state_dict can modify it\n    metadata = getattr(state_dict, "_metadata", None)\n    state_dict = state_dict.copy()\n    if metadata is not None:\n        state_dict._metadata = metadata\n\n    error_msgs = []\n\n    # PyTorch\'s `_load_from_state_dict` does not copy parameters in a module\'s descendants\n    # so we need to apply the function recursively.\n    def load(module: nn.Module, state_dict, prefix=""):\n        local_metadata = {} if metadata is None else metadata.get(prefix[:-1], {})\n        args = (state_dict, prefix, local_metadata, True, [], [], error_msgs)\n        # Parameters of module and children will start with prefix. We can exit early if there are none in this\n        # state_dict\n        if len([key for key in state_dict if key.startswith(prefix)]) > 0:\n            if is_deepspeed_zero3_enabled():\n                import deepspeed\n\n                # In sharded models, each shard has only part of the full state_dict, so only gather\n                # parameters that are in the current state_dict.\n                named_parameters = dict(module.named_parameters(prefix=prefix[:-1], recurse=False))\n                params_to_gather = [named_parameters[k] for k in state_dict.keys() if k in named_parameters]\n                if len(params_to_gather) > 0:\n                    # because zero3 puts placeholders in model params, this context\n                    # manager gathers (unpartitions) the params of the current layer, then loads from\n                    # the state dict and then re-partitions them again\n                    with deepspeed.zero.GatheredParameters(params_to_gather, modifier_rank=0):\n                        if torch.distributed.get_rank() == 0:\n                            module._load_from_state_dict(*args)\n            else:\n                module._load_from_state_dict(*args)\n\n        for name, child in module._modules.items():\n            if child is not None:\n                load(child, state_dict, prefix + name + ".")\n\n    load(model_to_load, state_dict, prefix=start_prefix)\n    # Delete `state_dict` so it could be collected by GC earlier. Note that `state_dict` is a copy of the argument, so\n    # it\'s safe to delete it.\n    del state_dict\n\n    return error_msgs\n\n\ndef find_submodule_and_param_name(model, long_key, start_prefix):\n    """\n    A helper util to find the last sub-module and the param/buffer name. If `start_prefix` is supplied it\'ll be removed\n    from the start of the key\n    """\n\n    if len(start_prefix) > 0 and long_key.startswith(start_prefix):\n        long_key = ".".join(long_key.split(".")[1:])\n\n    split_key = long_key.split(".")\n    submodule = model\n    while len(split_key) > 1:\n        if hasattr(submodule, split_key[0]):\n            submodule = getattr(submodule, split_key[0])\n            del split_key[0]\n        else:\n            submodule = None\n            break\n    if submodule == model:\n        submodule = None\n    return submodule, split_key[0]\n\n\ndef _move_model_to_meta(model, loaded_state_dict_keys, start_prefix):\n    """\n    Moves `loaded_state_dict_keys` in model to meta device which frees up the memory taken by those params.\n\n    `start_prefix` is used for models which insert their name into model keys, e.g. `bert` in\n    `bert.pooler.dense.weight`\n\n    """\n\n    # dematerialize param storage for keys that are going to be replaced by state_dict, by\n    # putting those on the meta device\n    for k in loaded_state_dict_keys:\n        submodule, param_name = find_submodule_and_param_name(model, k, start_prefix)\n        if submodule is not None:\n            # selectively switch to the meta device only those params/buffers that will\n            # be next replaced from state_dict. This a complex way to do p.to_("meta")\n            # since we have no in-place to_ for tensors.\n            new_val = getattr(submodule, param_name)\n            if isinstance(new_val, torch.nn.Parameter):\n                # isinstance returns False for Params on meta device, so switch after the check\n                new_val = torch.nn.Parameter(new_val.to("meta"))\n            else:\n                new_val = new_val.to("meta")\n            setattr(submodule, param_name, new_val)\n\n\ndef _load_state_dict_into_meta_model(\n    model,\n    state_dict,\n    loaded_state_dict_keys,  # left for now but could be removed, see below\n    start_prefix,\n    expected_keys,\n    device_map=None,\n    offload_folder=None,\n    offload_index=None,\n    state_dict_folder=None,\n    state_dict_index=None,\n    dtype=None,\n    is_quantized=False,\n    is_safetensors=False,\n    keep_in_fp32_modules=None,\n):\n    """\n    This is somewhat similar to `_load_state_dict_into_model`, but deals with a model that has some or all of its\n    params on a `meta` device. It replaces the model params with the data from the `state_dict`, while moving the\n    params back to the normal device, but only for `loaded_state_dict_keys`.\n\n    `start_prefix` is used for models which insert their name into model keys, e.g. `bert` in\n    `bert.pooler.dense.weight`\n\n    """\n\n    # XXX: remaining features to implement to be fully compatible with _load_state_dict_into_model\n    # - deepspeed zero 3 support\n    # - need to copy metadata if any - see _load_state_dict_into_model\n    # - handling error_msgs - mimicking the error handling in module._load_from_state_dict()\n    # - Is there a situation where some keys aren\'t in `loaded_state_dict_keys` and in which case\n    #   they won\'t get loaded.\n\n    if is_quantized:\n        from .integrations import set_module_quantized_tensor_to_device\n\n    error_msgs = []\n\n    old_keys = []\n    new_keys = []\n    for key in state_dict.keys():\n        new_key = None\n        if "gamma" in key:\n            new_key = key.replace("gamma", "weight")\n        if "beta" in key:\n            new_key = key.replace("beta", "bias")\n        if new_key:\n            old_keys.append(key)\n            new_keys.append(new_key)\n    for old_key, new_key in zip(old_keys, new_keys):\n        state_dict[new_key] = state_dict.pop(old_key)\n\n    for param_name, param in state_dict.items():\n        # First part of the test is always true as load_state_dict_keys always contains state_dict keys.\n        if param_name not in loaded_state_dict_keys or param_name not in expected_keys:\n            continue\n\n        if param_name.startswith(start_prefix):\n            param_name = param_name[len(start_prefix) :]\n\n        module_name = param_name\n        set_module_kwargs = {}\n\n        # We convert floating dtypes to the `dtype` passed. We want to keep the buffers/params\n        # in int/uint/bool and not cast them.\n        if dtype is not None and torch.is_floating_point(param):\n            if (\n                keep_in_fp32_modules is not None\n                and any(\n                    module_to_keep_in_fp32 in param_name.split(".") for module_to_keep_in_fp32 in keep_in_fp32_modules\n                )\n                and dtype == torch.float16\n            ):\n                param = param.to(torch.float32)\n\n                # For backward compatibility with older versions of `accelerate`\n                # TODO: @sgugger replace this check with version check at the next `accelerate` release\n                if "dtype" in list(inspect.signature(set_module_tensor_to_device).parameters):\n                    set_module_kwargs["dtype"] = torch.float32\n            else:\n                param = param.to(dtype)\n\n        # For compatibility with PyTorch load_state_dict which converts state dict dtype to existing dtype in model\n        if dtype is None:\n            old_param = model\n            splits = param_name.split(".")\n            for split in splits:\n                old_param = getattr(old_param, split)\n                if old_param is None:\n                    break\n\n            if old_param is not None:\n                param = param.to(old_param.dtype)\n\n        set_module_kwargs["value"] = param\n\n        if device_map is None:\n            param_device = "cpu"\n        else:\n            # find next higher level module that is defined in device_map:\n            # bert.lm_head.weight -> bert.lm_head -> bert -> \'\'\n            while len(module_name) > 0 and module_name not in device_map:\n                module_name = ".".join(module_name.split(".")[:-1])\n            if module_name == "" and "" not in device_map:\n                # TODO: group all errors and raise at the end.\n                raise ValueError(f"{param_name} doesn\'t have any device set.")\n            param_device = device_map[module_name]\n\n        if param_device == "disk":\n            if not is_safetensors:\n                offload_index = offload_weight(param, param_name, offload_folder, offload_index)\n        elif param_device == "cpu" and state_dict_index is not None:\n            state_dict_index = offload_weight(param, param_name, state_dict_folder, state_dict_index)\n        elif not is_quantized:\n            # For backward compatibility with older versions of `accelerate`\n            set_module_tensor_to_device(model, param_name, param_device, **set_module_kwargs)\n        else:\n            if param.dtype == torch.int8 and param_name.replace("weight", "SCB") in state_dict.keys():\n                fp16_statistics = state_dict[param_name.replace("weight", "SCB")]\n            else:\n                fp16_statistics = None\n\n            if "SCB" not in param_name:\n                set_module_quantized_tensor_to_device(\n                    model, param_name, param_device, value=param, fp16_statistics=fp16_statistics\n                )\n\n    return error_msgs, offload_index, state_dict_index\n\n\ndef _add_variant(weights_name: str, variant: Optional[str] = None) -> str:\n    if variant is not None:\n        splits = weights_name.split(".")\n        splits = splits[:-1] + [variant] + splits[-1:]\n        weights_name = ".".join(splits)\n\n    return weights_name\n\n\nclass ModuleUtilsMixin:\n    """\n    A few utilities for `torch.nn.Modules`, to be used as a mixin.\n    """\n\n    @staticmethod\n    def _hook_rss_memory_pre_forward(module, *args, **kwargs):\n        try:\n            import psutil\n        except ImportError:\n            raise ImportError("You need to install psutil (pip install psutil) to use memory tracing.")\n\n        process = psutil.Process(os.getpid())\n        mem = process.memory_info()\n        module.mem_rss_pre_forward = mem.rss\n        return None\n\n    @staticmethod\n    def _hook_rss_memory_post_forward(module, *args, **kwargs):\n        try:\n            import psutil\n        except ImportError:\n            raise ImportError("You need to install psutil (pip install psutil) to use memory tracing.")\n\n        process = psutil.Process(os.getpid())\n        mem = process.memory_info()\n        module.mem_rss_post_forward = mem.rss\n        mem_rss_diff = module.mem_rss_post_forward - module.mem_rss_pre_forward\n        module.mem_rss_diff = mem_rss_diff + (module.mem_rss_diff if hasattr(module, "mem_rss_diff") else 0)\n        return None\n\n    def add_memory_hooks(self):\n        """\n        Add a memory hook before and after each sub-module forward pass to record increase in memory consumption.\n\n        Increase in memory consumption is stored in a `mem_rss_diff` attribute for each module and can be reset to zero\n        with `model.reset_memory_hooks_state()`.\n        """\n        for module in self.modules():\n            module.register_forward_pre_hook(self._hook_rss_memory_pre_forward)\n            module.register_forward_hook(self._hook_rss_memory_post_forward)\n        self.reset_memory_hooks_state()\n\n    def reset_memory_hooks_state(self):\n        """\n        Reset the `mem_rss_diff` attribute of each module (see [`~modeling_utils.ModuleUtilsMixin.add_memory_hooks`]).\n        """\n        for module in self.modules():\n            module.mem_rss_diff = 0\n            module.mem_rss_post_forward = 0\n            module.mem_rss_pre_forward = 0\n\n    @property\n    def device(self) -> torch.device:\n        """\n        `torch.device`: The device on which the module is (assuming that all the module parameters are on the same\n        device).\n        """\n        return get_parameter_device(self)\n\n    @property\n    def dtype(self) -> torch.dtype:\n        """\n        `torch.dtype`: The dtype of the module (assuming that all the module parameters have the same dtype).\n        """\n        return get_parameter_dtype(self)\n\n    def invert_attention_mask(self, encoder_attention_mask: Tensor) -> Tensor:\n        """\n        Invert an attention mask (e.g., switches 0. and 1.).\n\n        Args:\n            encoder_attention_mask (`torch.Tensor`): An attention mask.\n\n        Returns:\n            `torch.Tensor`: The inverted attention mask.\n        """\n        if encoder_attention_mask.dim() == 3:\n            encoder_extended_attention_mask = encoder_attention_mask[:, None, :, :]\n        if encoder_attention_mask.dim() == 2:\n            encoder_extended_attention_mask = encoder_attention_mask[:, None, None, :]\n        # T5 has a mask that can compare sequence ids, we can simulate this here with this transposition\n        # Cf. https://github.com/tensorflow/mesh/blob/8d2465e9bc93129b913b5ccc6a59aa97abd96ec6/mesh_tensorflow\n        # /transformer/transformer_layers.py#L270\n        # encoder_extended_attention_mask = (encoder_extended_attention_mask ==\n        # encoder_extended_attention_mask.transpose(-1, -2))\n        encoder_extended_attention_mask = encoder_extended_attention_mask.to(dtype=self.dtype)  # fp16 compatibility\n        encoder_extended_attention_mask = (1.0 - encoder_extended_attention_mask) * torch.finfo(self.dtype).min\n\n        return encoder_extended_attention_mask\n\n    @staticmethod\n    def create_extended_attention_mask_for_decoder(input_shape, attention_mask, device=None):\n        if device is not None:\n            warnings.warn(\n                "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning\n            )\n        else:\n            device = attention_mask.device\n        batch_size, seq_length = input_shape\n        seq_ids = torch.arange(seq_length, device=device)\n        causal_mask = seq_ids[None, None, :].repeat(batch_size, seq_length, 1) <= seq_ids[None, :, None]\n        # in case past_key_values are used we need to add a prefix ones mask to the causal mask\n        # causal and attention masks must have same type with pytorch version < 1.3\n        causal_mask = causal_mask.to(attention_mask.dtype)\n\n        if causal_mask.shape[1] < attention_mask.shape[1]:\n            prefix_seq_len = attention_mask.shape[1] - causal_mask.shape[1]\n            causal_mask = torch.cat(\n                [\n                    torch.ones((batch_size, seq_length, prefix_seq_len), device=device, dtype=causal_mask.dtype),\n                    causal_mask,\n                ],\n                axis=-1,\n            )\n\n        extended_attention_mask = causal_mask[:, None, :, :] * attention_mask[:, None, None, :]\n        return extended_attention_mask\n\n    def get_extended_attention_mask(\n        self, attention_mask: Tensor, input_shape: Tuple[int], device: torch.device = None, dtype: torch.float = None\n    ) -> Tensor:\n        """\n        Makes broadcastable attention and causal masks so that future and masked tokens are ignored.\n\n        Arguments:\n            attention_mask (`torch.Tensor`):\n                Mask with ones indicating tokens to attend to, zeros for tokens to ignore.\n            input_shape (`Tuple[int]`):\n                The shape of the input to the model.\n\n        Returns:\n            `torch.Tensor` The extended attention mask, with a the same dtype as `attention_mask.dtype`.\n        """\n        if dtype is None:\n            dtype = self.dtype\n\n        if not (attention_mask.dim() == 2 and self.config.is_decoder):\n            # show warning only if it won\'t be shown in `create_extended_attention_mask_for_decoder`\n            if device is not None:\n                warnings.warn(\n                    "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning\n                )\n        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]\n        # ourselves in which case we just need to make it broadcastable to all heads.\n        if attention_mask.dim() == 3:\n            extended_attention_mask = attention_mask[:, None, :, :]\n        elif attention_mask.dim() == 2:\n            # Provided a padding mask of dimensions [batch_size, seq_length]\n            # - if the model is a decoder, apply a causal mask in addition to the padding mask\n            # - if the model is an encoder, make the mask broadcastable to [batch_size, num_heads, seq_length, seq_length]\n            if self.config.is_decoder:\n                extended_attention_mask = ModuleUtilsMixin.create_extended_attention_mask_for_decoder(\n                    input_shape, attention_mask, device\n                )\n            else:\n                extended_attention_mask = attention_mask[:, None, None, :]\n        else:\n            raise ValueError(\n                f"Wrong shape for input_ids (shape {input_shape}) or attention_mask (shape {attention_mask.shape})"\n            )\n\n        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for\n        # masked positions, this operation will create a tensor which is 0.0 for\n        # positions we want to attend and the dtype\'s smallest value for masked positions.\n        # Since we are adding it to the raw scores before the softmax, this is\n        # effectively the same as removing these entirely.\n        extended_attention_mask = extended_attention_mask.to(dtype=dtype)  # fp16 compatibility\n        extended_attention_mask = (1.0 - extended_attention_mask) * torch.finfo(dtype).min\n        return extended_attention_mask\n\n    def get_head_mask(\n        self, head_mask: Optional[Tensor], num_hidden_layers: int, is_attention_chunked: bool = False\n    ) -> Tensor:\n        """\n        Prepare the head mask if needed.\n\n        Args:\n            head_mask (`torch.Tensor` with shape `[num_heads]` or `[num_hidden_layers x num_heads]`, *optional*):\n                The mask indicating if we should keep the heads or not (1.0 for keep, 0.0 for discard).\n            num_hidden_layers (`int`):\n                The number of hidden layers in the model.\n            is_attention_chunked (`bool`, *optional*, defaults to `False`):\n                Whether or not the attentions scores are computed by chunks or not.\n\n        Returns:\n            `torch.Tensor` with shape `[num_hidden_layers x batch x num_heads x seq_length x seq_length]` or list with\n            `[None]` for each layer.\n        """\n        if head_mask is not None:\n            head_mask = self._convert_head_mask_to_5d(head_mask, num_hidden_layers)\n            if is_attention_chunked is True:\n                head_mask = head_mask.unsqueeze(-1)\n        else:\n            head_mask = [None] * num_hidden_layers\n\n        return head_mask\n\n    def _convert_head_mask_to_5d(self, head_mask, num_hidden_layers):\n        """-> [num_hidden_layers x batch x num_heads x seq_length x seq_length]"""\n        if head_mask.dim() == 1:\n            head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)\n            head_mask = head_mask.expand(num_hidden_layers, -1, -1, -1, -1)\n        elif head_mask.dim() == 2:\n            head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)  # We can specify head_mask for each layer\n        assert head_mask.dim() == 5, f"head_mask.dim != 5, instead {head_mask.dim()}"\n        head_mask = head_mask.to(dtype=self.dtype)  # switch to float if need + fp16 compatibility\n        return head_mask\n\n    def num_parameters(self, only_trainable: bool = False, exclude_embeddings: bool = False) -> int:\n        """\n        Get number of (optionally, trainable or non-embeddings) parameters in the module.\n\n        Args:\n            only_trainable (`bool`, *optional*, defaults to `False`):\n                Whether or not to return only the number of trainable parameters\n\n            exclude_embeddings (`bool`, *optional*, defaults to `False`):\n                Whether or not to return only the number of non-embeddings parameters\n\n        Returns:\n            `int`: The number of parameters.\n        """\n\n        if exclude_embeddings:\n            embedding_param_names = [\n                f"{name}.weight" for name, module_type in self.named_modules() if isinstance(module_type, nn.Embedding)\n            ]\n            total_parameters = [\n                parameter for name, parameter in self.named_parameters() if name not in embedding_param_names\n            ]\n        else:\n            total_parameters = list(self.parameters())\n\n        total_numel = []\n        is_loaded_in_4bit = getattr(self, "is_loaded_in_4bit", False)\n        if is_loaded_in_4bit:\n            if is_bitsandbytes_available():\n                import bitsandbytes as bnb\n            else:\n                raise ValueError(\n                    "bitsandbytes is not installed but it seems that the model has been loaded in 4bit precision, something went wrong"\n                    " make sure to install bitsandbytes with `pip install bitsandbytes`."\n                )\n\n        for param in total_parameters:\n            if param.requires_grad or not only_trainable:\n                # For 4bit models, we need to multiply the number of parameters by 2 as half of the parameters are\n                # used for the 4bit quantization (uint8 tensors are stored)\n                if is_loaded_in_4bit and isinstance(param, bnb.nn.Params4bit):\n                    total_numel.append(param.numel() * 2)\n                else:\n                    total_numel.append(param.numel())\n\n        return sum(total_numel)\n\n    def estimate_tokens(self, input_dict: Dict[str, Union[torch.Tensor, Any]]) -> int:\n        """\n        Helper function to estimate the total number of tokens from the model inputs.\n\n        Args:\n            inputs (`dict`): The model inputs.\n\n        Returns:\n            `int`: The total number of tokens.\n        """\n        if not hasattr(self, "warnings_issued"):\n            self.warnings_issued = {}\n        if self.main_input_name in input_dict:\n            return input_dict[self.main_input_name].numel()\n        elif "estimate_tokens" not in self.warnings_issued:\n            logger.warning(\n                "Could not estimate the number of tokens of the input, floating-point operations will not be computed"\n            )\n            self.warnings_issued["estimate_tokens"] = True\n        return 0\n\n    def floating_point_ops(\n        self, input_dict: Dict[str, Union[torch.Tensor, Any]], exclude_embeddings: bool = True\n    ) -> int:\n        """\n        Get number of (optionally, non-embeddings) floating-point operations for the forward and backward passes of a\n        batch with this transformer model. Default approximation neglects the quadratic dependency on the number of\n        tokens (valid if `12 * d_model << sequence_length`) as laid out in [this\n        paper](https://arxiv.org/pdf/2001.08361.pdf) section 2.1. Should be overridden for transformers with parameter\n        re-use e.g. Albert or Universal Transformers, or if doing long-range modeling with very high sequence lengths.\n\n        Args:\n            batch_size (`int`):\n                The batch size for the forward pass.\n\n            sequence_length (`int`):\n                The number of tokens in each line of the batch.\n\n            exclude_embeddings (`bool`, *optional*, defaults to `True`):\n                Whether or not to count embedding and softmax operations.\n\n        Returns:\n            `int`: The number of floating-point operations.\n        """\n\n        return 6 * self.estimate_tokens(input_dict) * self.num_parameters(exclude_embeddings=exclude_embeddings)\n\n\nclass PreTrainedModel(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin, PeftAdapterMixin):\n    r"""\n    Base class for all models.\n\n    [`PreTrainedModel`] takes care of storing the configuration of the models and handles methods for loading,\n    downloading and saving models as well as a few methods common to all models to:\n\n        - resize the input embeddings,\n        - prune heads in the self-attention heads.\n\n    Class attributes (overridden by derived classes):\n\n        - **config_class** ([`PretrainedConfig`]) -- A subclass of [`PretrainedConfig`] to use as configuration class\n          for this model architecture.\n        - **load_tf_weights** (`Callable`) -- A python *method* for loading a TensorFlow checkpoint in a PyTorch model,\n          taking as arguments:\n\n            - **model** ([`PreTrainedModel`]) -- An instance of the model on which to load the TensorFlow checkpoint.\n            - **config** ([`PreTrainedConfig`]) -- An instance of the configuration associated to the model.\n            - **path** (`str`) -- A path to the TensorFlow checkpoint.\n\n        - **base_model_prefix** (`str`) -- A string indicating the attribute associated to the base model in derived\n          classes of the same architecture adding modules on top of the base model.\n        - **is_parallelizable** (`bool`) -- A flag indicating whether this model supports model parallelization.\n        - **main_input_name** (`str`) -- The name of the principal input to the model (often `input_ids` for NLP\n          models, `pixel_values` for vision models and `input_values` for speech models).\n    """\n\n    config_class = None\n    base_model_prefix = ""\n    main_input_name = "input_ids"\n    _auto_class = None\n    _no_split_modules = None\n    _skip_keys_device_placement = None\n    _keep_in_fp32_modules = None\n\n    # a list of `re` patterns of `state_dict` keys that should be removed from the list of missing\n    # keys we find (keys inside the model but not in the checkpoint) and avoid unnecessary warnings.\n    _keys_to_ignore_on_load_missing = None\n    # a list of `re` patterns of `state_dict` keys that should be removed from the list of\n    # unexpected keys we find (keys inside the checkpoint but not the model) and avoid unnecessary\n    # warnings.\n    _keys_to_ignore_on_load_unexpected = None\n    # a list of `state_dict` keys to ignore when saving the model (useful for keys that aren\'t\n    # trained, but which are either deterministic or tied variables)\n    _keys_to_ignore_on_save = None\n    # a list of `state_dict` keys that are potentially tied to another key in the state_dict.\n    _tied_weights_keys = None\n\n    is_parallelizable = False\n    supports_gradient_checkpointing = False\n\n    # Flash Attention 2 support\n    _supports_flash_attn_2 = False\n\n    @property\n    def dummy_inputs(self) -> Dict[str, torch.Tensor]:\n        """\n        `Dict[str, torch.Tensor]`: Dummy inputs to do a forward pass in the network.\n        """\n        return {"input_ids": torch.tensor(DUMMY_INPUTS)}\n\n    @property\n    def framework(self) -> str:\n        """\n        :str: Identifies that this is a PyTorch model.\n        """\n        return "pt"\n\n    def __init__(self, config: PretrainedConfig, *inputs, **kwargs):\n        super().__init__()\n        if not isinstance(config, PretrainedConfig):\n            raise ValueError(\n                f"Parameter config in `{self.__class__.__name__}(config)` should be an instance of class "\n                "`PretrainedConfig`. To create a model from a pretrained model use "\n                f"`model = {self.__class__.__name__}.from_pretrained(PRETRAINED_MODEL_NAME)`"\n            )\n        # Save config and origin of the pretrained weights if given in model\n        self.config = config\n        self.name_or_path = config.name_or_path\n        self.warnings_issued = {}\n        self.generation_config = GenerationConfig.from_model_config(config) if self.can_generate() else None\n\n    def post_init(self):\n        """\n        A method executed at the end of each Transformer model initialization, to execute code that needs the model\'s\n        modules properly initialized (such as weight initialization).\n        """\n        self.init_weights()\n        self._backward_compatibility_gradient_checkpointing()\n\n    def _backward_compatibility_gradient_checkpointing(self):\n        if self.supports_gradient_checkpointing and getattr(self.config, "gradient_checkpointing", False):\n            self.gradient_checkpointing_enable()\n            # Remove the attribute now that is has been consumed, so it\'s no saved in the config.\n            delattr(self.config, "gradient_checkpointing")\n\n    @classmethod\n    def _from_config(cls, config, **kwargs):\n        """\n        All context managers that the model should be initialized under go here.\n\n        Args:\n            torch_dtype (`torch.dtype`, *optional*):\n                Override the default `torch.dtype` and load the model under this dtype.\n            use_flash_attention_2 (`bool`, *optional*):\n                Whether to load the model with Flash Attention 2 modules.\n        """\n        torch_dtype = kwargs.pop("torch_dtype", None)\n        use_flash_attention_2 = kwargs.pop("use_flash_attention_2", False)\n\n        # override default dtype if needed\n        dtype_orig = None\n        if torch_dtype is not None:\n            dtype_orig = cls._set_default_torch_dtype(torch_dtype)\n\n        if use_flash_attention_2:\n            config = cls._check_and_enable_flash_attn_2(config, torch_dtype)\n\n        if is_deepspeed_zero3_enabled():\n            import deepspeed\n\n            logger.info("Detected DeepSpeed ZeRO-3: activating zero.init() for this model")\n            # this immediately partitions the model across all gpus, to avoid the overhead in time\n            # and memory copying it on CPU or each GPU first\n            with deepspeed.zero.Init(config_dict_or_path=deepspeed_config()):\n                model = cls(config, **kwargs)\n        else:\n            model = cls(config, **kwargs)\n\n        # restore default dtype if it was modified\n        if dtype_orig is not None:\n            torch.set_default_dtype(dtype_orig)\n\n        return model\n\n    @classmethod\n    def _set_default_torch_dtype(cls, dtype: torch.dtype) -> torch.dtype:\n        """\n        Change the default dtype and return the previous one. This is needed when wanting to instantiate the model\n        under specific dtype.\n\n        Args:\n            dtype (`torch.dtype`):\n                a floating dtype to set to.\n\n        Returns:\n            `torch.dtype`: the original `dtype` that can be used to restore `torch.set_default_dtype(dtype)` if it was\n            modified. If it wasn\'t, returns `None`.\n\n        Note `set_default_dtype` currently only works with floating-point types and asserts if for example,\n        `torch.int64` is passed. So if a non-float `dtype` is passed this functions will throw an exception.\n        """\n        if not dtype.is_floating_point:\n            raise ValueError(\n                f"Can\'t instantiate {cls.__name__} model under dtype={dtype} since it is not a floating point dtype"\n            )\n\n        logger.info(f"Instantiating {cls.__name__} model under default dtype {dtype}.")\n        dtype_orig = torch.get_default_dtype()\n        torch.set_default_dtype(dtype)\n        return dtype_orig\n\n    @property\n    def base_model(self) -> nn.Module:\n        """\n        `torch.nn.Module`: The main body of the model.\n        """\n        return getattr(self, self.base_model_prefix, self)\n\n    @classmethod\n    def can_generate(cls) -> bool:\n        """\n        Returns whether this model can generate sequences with `.generate()`.\n\n        Returns:\n            `bool`: Whether this model can generate sequences with `.generate()`.\n        """\n        # Detects whether `prepare_inputs_for_generation` has been overwritten, which is a requirement for generation.\n        # Alternativelly, the model can also have a custom `generate` function.\n        if "GenerationMixin" in str(cls.prepare_inputs_for_generation) and "GenerationMixin" in str(cls.generate):\n            return False\n        return True\n\n    @classmethod\n    def _check_and_enable_flash_attn_2(\n        cls, config, torch_dtype: Optional[torch.dtype] = None, device_map: Optional[Union[str, Dict[str, int]]] = None\n    ) -> PretrainedConfig:\n        """\n        If you don\'t know about Flash Attention, check out the official repository of flash attention:\n        https://github.com/Dao-AILab/flash-attention\n\n        For using Flash Attention 1.0 you can do it directly via the `BetterTransformer` API, have a look at this\n        specific section of the documentation to learn more about it:\n        https://huggingface.co/docs/transformers/main/en/perf_infer_gpu_one#decoder-models\n\n        The method checks if the current setup is compatible with Flash Attention as it requires the model to be in\n        half precision and not ran on CPU.\n\n        If all checks pass, the method will create an attribute in the config `_flash_attn_2_enabled` so that the model\n        can initialize the correct attention module\n        """\n        if not cls._supports_flash_attn_2:\n            raise ValueError(\n                "The current architecture does not support Flash Attention 2.0. Please open an issue on GitHub to "\n                "request support for this architecture: https://github.com/huggingface/transformers/issues/new"\n            )\n\n        if not is_flash_attn_2_available():\n            raise ImportError(\n                "Flash Attention 2 is not available. Please refer to the documentation of https://github.com/Dao-AILab/flash-attention for"\n                " installing it. Make sure to have at least the version 2.1.0"\n            )\n        else:\n            flash_attention_version = version.parse(importlib.metadata.version("flash_attn"))\n            is_flash_greater_than_2 = flash_attention_version >= version.parse("2.1.0")\n            if not is_flash_greater_than_2:\n                raise ValueError(\n                    f"You need flash_attn package version to be greater or equal than 2.1. Make sure to have that version installed - detected version {flash_attention_version}"\n                )\n\n        _is_bettertransformer = getattr(cls, "use_bettertransformer", False)\n\n        if _is_bettertransformer:\n            raise ValueError(\n                "Flash Attention 2 and BetterTransformer API are not compatible. Please make sure to disable BetterTransformers by doing model.reverse_bettertransformer()"\n            )\n\n        if torch_dtype is None:\n            logger.warning(\n                "You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour"\n            )\n        elif torch_dtype is not None and torch_dtype not in [torch.float16, torch.bfloat16]:\n            raise ValueError(\n                f"Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes. You passed {torch_dtype}, this might lead to"\n                " unexpected behaviour."\n            )\n\n        if device_map is None:\n            if torch.cuda.is_available():\n                logger.warning(\n                    "You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU"\n                    " after initializing it on CPU with `model.to(\'cuda\')`."\n                )\n            else:\n                raise ValueError(\n                    "You are attempting to use Flash Attention 2.0 with a model initialized on CPU and with no GPU available. "\n                    "This is not supported yet. Please make sure to have access to a GPU and either initialise the model on a GPU by passing a device_map "\n                    "or initialising the model on CPU and then moving it to GPU."\n                )\n        elif (\n            device_map is not None\n            and isinstance(device_map, dict)\n            and ("cpu" in device_map.values() or "disk" in device_map.values())\n        ):\n            raise ValueError(\n                "You are attempting to use Flash Attention 2.0 with a model dispatched on CPU or disk. This is not supported. Please make sure to "\n                "initialise the model on a GPU by passing a device_map that contains only GPU devices as keys."\n            )\n        config._flash_attn_2_enabled = True\n        return config\n\n    def enable_input_require_grads(self):\n        """\n        Enables the gradients for the input embeddings. This is useful for fine-tuning adapter weights while keeping\n        the model weights fixed.\n        """\n\n        def make_inputs_require_grads(module, input, output):\n            output.requires_grad_(True)\n\n        self._require_grads_hook = self.get_input_embeddings().register_forward_hook(make_inputs_require_grads)\n\n    def disable_input_require_grads(self):\n        """\n        Removes the `_require_grads_hook`.\n        """\n        self._require_grads_hook.remove()\n\n    def get_input_embeddings(self) -> nn.Module:\n        """\n        Returns the model\'s input embeddings.\n\n        Returns:\n            `nn.Module`: A torch module mapping vocabulary to hidden states.\n        """\n        base_model = getattr(self, self.base_model_prefix, self)\n        if base_model is not self:\n            return base_model.get_input_embeddings()\n        else:\n            raise NotImplementedError\n\n    def set_input_embeddings(self, value: nn.Module):\n        """\n        Set model\'s input embeddings.\n\n        Args:\n            value (`nn.Module`): A module mapping vocabulary to hidden states.\n        """\n        base_model = getattr(self, self.base_model_prefix, self)\n        if base_model is not self:\n            base_model.set_input_embeddings(value)\n        else:\n            raise NotImplementedError\n\n    def get_output_embeddings(self) -> nn.Module:\n        """\n        Returns the model\'s output embeddings.\n\n        Returns:\n            `nn.Module`: A torch module mapping hidden states to vocabulary.\n        """\n        return None  # Overwrite for models with output embeddings\n\n    def _init_weights(self, module):\n        """\n        Initialize the weights. This method should be overridden by derived class.\n        """\n        pass\n\n    def _initialize_weights(self, module):\n        """\n        Initialize the weights if they are not already initialized.\n        """\n        if getattr(module, "_is_hf_initialized", False):\n            return\n        self._init_weights(module)\n        module._is_hf_initialized = True\n\n    def tie_weights(self):\n        """\n        Tie the weights between the input embeddings and the output embeddings.\n\n        If the `torchscript` flag is set in the configuration, can\'t handle parameter sharing so we are cloning the\n        weights instead.\n        """\n        if getattr(self.config, "tie_word_embeddings", True):\n            output_embeddings = self.get_output_embeddings()\n            if output_embeddings is not None:\n                self._tie_or_clone_weights(output_embeddings, self.get_input_embeddings())\n\n        if getattr(self.config, "is_encoder_decoder", False) and getattr(self.config, "tie_encoder_decoder", False):\n            if hasattr(self, self.base_model_prefix):\n                self = getattr(self, self.base_model_prefix)\n            self._tie_encoder_decoder_weights(self.encoder, self.decoder, self.base_model_prefix)\n\n        for module in self.modules():\n            if hasattr(module, "_tie_weights"):\n                module._tie_weights()\n\n    @staticmethod\n    def _tie_encoder_decoder_weights(encoder: nn.Module, decoder: nn.Module, base_model_prefix: str):\n        uninitialized_encoder_weights: List[str] = []\n        if decoder.__class__ != encoder.__class__:\n            logger.info(\n                f"{decoder.__class__} and {encoder.__class__} are not equal. In this case make sure that all encoder"\n                " weights are correctly initialized."\n            )\n\n        def tie_encoder_to_decoder_recursively(\n            decoder_pointer: nn.Module,\n            encoder_pointer: nn.Module,\n            module_name: str,\n            uninitialized_encoder_weights: List[str],\n            depth=0,\n        ):\n            assert isinstance(decoder_pointer, nn.Module) and isinstance(\n                encoder_pointer, nn.Module\n            ), f"{decoder_pointer} and {encoder_pointer} have to be of type nn.Module"\n            if hasattr(decoder_pointer, "weight"):\n                assert hasattr(encoder_pointer, "weight")\n                encoder_pointer.weight = decoder_pointer.weight\n                if hasattr(decoder_pointer, "bias"):\n                    assert hasattr(encoder_pointer, "bias")\n                    encoder_pointer.bias = decoder_pointer.bias\n                return\n\n            encoder_modules = encoder_pointer._modules\n            decoder_modules = decoder_pointer._modules\n            if len(decoder_modules) > 0:\n                assert (\n                    len(encoder_modules) > 0\n                ), f"Encoder module {encoder_pointer} does not match decoder module {decoder_pointer}"\n\n                all_encoder_weights = {module_name + "/" + sub_name for sub_name in encoder_modules.keys()}\n                encoder_layer_pos = 0\n                for name, module in decoder_modules.items():\n                    if name.isdigit():\n                        encoder_name = str(int(name) + encoder_layer_pos)\n                        decoder_name = name\n                        if not isinstance(decoder_modules[decoder_name], type(encoder_modules[encoder_name])) and len(\n                            encoder_modules\n                        ) != len(decoder_modules):\n                            # this can happen if the name corresponds to the position in a list module list of layers\n                            # in this case the decoder has added a cross-attention that the encoder does not have\n                            # thus skip this step and subtract one layer pos from encoder\n                            encoder_layer_pos -= 1\n                            continue\n                    elif name not in encoder_modules:\n                        continue\n                    elif depth > 500:\n                        raise ValueError(\n                            "Max depth of recursive function `tie_encoder_to_decoder` reached. It seems that there is"\n                            " a circular dependency between two or more `nn.Modules` of your model."\n                        )\n                    else:\n                        decoder_name = encoder_name = name\n                    tie_encoder_to_decoder_recursively(\n                        decoder_modules[decoder_name],\n                        encoder_modules[encoder_name],\n                        module_name + "/" + name,\n                        uninitialized_encoder_weights,\n                        depth=depth + 1,\n                    )\n                    all_encoder_weights.remove(module_name + "/" + encoder_name)\n\n                uninitialized_encoder_weights += list(all_encoder_weights)\n\n        # tie weights recursively\n        tie_encoder_to_decoder_recursively(decoder, encoder, base_model_prefix, uninitialized_encoder_weights)\n        if len(uninitialized_encoder_weights) > 0:\n            logger.warning(\n                f"The following encoder weights were not tied to the decoder {uninitialized_encoder_weights}"\n            )\n\n    def _tie_or_clone_weights(self, output_embeddings, input_embeddings):\n        """Tie or clone module weights depending of whether we are using TorchScript or not"""\n        if self.config.torchscript:\n            output_embeddings.weight = nn.Parameter(input_embeddings.weight.clone())\n        else:\n            output_embeddings.weight = input_embeddings.weight\n\n        if getattr(output_embeddings, "bias", None) is not None:\n            output_embeddings.bias.data = nn.functional.pad(\n                output_embeddings.bias.data,\n                (\n                    0,\n                    output_embeddings.weight.shape[0] - output_embeddings.bias.shape[0],\n                ),\n                "constant",\n                0,\n            )\n        if hasattr(output_embeddings, "out_features") and hasattr(input_embeddings, "num_embeddings"):\n            output_embeddings.out_features = input_embeddings.num_embeddings\n\n    def _get_no_split_modules(self, device_map: str):\n        """\n        Get the modules of the model that should not be spit when using device_map. We iterate through the modules to\n        get the underlying `_no_split_modules`.\n\n        Args:\n            device_map (`str`):\n                The device map value. Options are ["auto", "balanced", "balanced_low_0", "sequential"]\n\n        Returns:\n            `List[str]`: List of modules that should not be split\n        """\n        _no_split_modules = set()\n        modules_to_check = [self]\n        while len(modules_to_check) > 0:\n            module = modules_to_check.pop(-1)\n            # if the module does not appear in _no_split_modules, we also check the children\n            if module.__class__.__name__ not in _no_split_modules:\n                if isinstance(module, PreTrainedModel):\n                    if module._no_split_modules is None:\n                        raise ValueError(\n                            f"{module.__class__.__name__} does not support `device_map=\'{device_map}\'`. To implement support, the model "\n                            "class needs to implement the `_no_split_modules` attribute."\n                        )\n                    else:\n                        _no_split_modules = _no_split_modules | set(module._no_split_modules)\n                modules_to_check += list(module.children())\n        return list(_no_split_modules)\n\n    def resize_token_embeddings(\n        self, new_num_tokens: Optional[int] = None, pad_to_multiple_of: Optional[int] = None\n    ) -> nn.Embedding:\n        """\n        Resizes input token embeddings matrix of the model if `new_num_tokens != config.vocab_size`.\n\n        Takes care of tying weights embeddings afterwards if the model class has a `tie_weights()` method.\n\n        Arguments:\n            new_num_tokens (`int`, *optional*):\n                The number of new tokens in the embedding matrix. Increasing the size will add newly initialized\n                vectors at the end. Reducing the size will remove vectors from the end. If not provided or `None`, just\n                returns a pointer to the input tokens `torch.nn.Embedding` module of the model without doing anything.\n            pad_to_multiple_of (`int`, *optional*):\n                If set will pad the embedding matrix to a multiple of the provided value.If `new_num_tokens` is set to\n                `None` will just pad the embedding to a multiple of `pad_to_multiple_of`.\n\n                This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability\n                `>= 7.5` (Volta), or on TPUs which benefit from having sequence lengths be a multiple of 128. For more\n                details about this, or help on choosing the correct value for resizing, refer to this guide:\n                https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc\n\n        Return:\n            `torch.nn.Embedding`: Pointer to the input tokens Embeddings Module of the model.\n        """\n        model_embeds = self._resize_token_embeddings(new_num_tokens, pad_to_multiple_of)\n        if new_num_tokens is None and pad_to_multiple_of is None:\n            return model_embeds\n\n        # Update base model and current model config\n        self.config.vocab_size = model_embeds.weight.shape[0]\n        self.vocab_size = model_embeds.weight.shape[0]\n\n        # Tie weights again if needed\n        self.tie_weights()\n\n        return model_embeds\n\n    def _resize_token_embeddings(self, new_num_tokens, pad_to_multiple_of=None):\n        old_embeddings = self.get_input_embeddings()\n        new_embeddings = self._get_resized_embeddings(old_embeddings, new_num_tokens, pad_to_multiple_of)\n        if hasattr(old_embeddings, "_hf_hook"):\n            hook = old_embeddings._hf_hook\n            add_hook_to_module(new_embeddings, hook)\n        old_embeddings_requires_grad = old_embeddings.weight.requires_grad\n        new_embeddings.requires_grad_(old_embeddings_requires_grad)\n        self.set_input_embeddings(new_embeddings)\n\n        # Update new_num_tokens with the actual size of new_embeddings\n        if pad_to_multiple_of is not None:\n            if is_deepspeed_zero3_enabled():\n                import deepspeed\n\n                with deepspeed.zero.GatheredParameters(new_embeddings.weight, modifier_rank=None):\n                    new_num_tokens = new_embeddings.weight.shape[0]\n            else:\n                new_num_tokens = new_embeddings.weight.shape[0]\n\n        # if word embeddings are not tied, make sure that lm head is resized as well\n        if self.get_output_embeddings() is not None and not self.config.tie_word_embeddings:\n            old_lm_head = self.get_output_embeddings()\n            new_lm_head = self._get_resized_lm_head(old_lm_head, new_num_tokens)\n            if hasattr(old_lm_head, "_hf_hook"):\n                hook = old_lm_head._hf_hook\n                add_hook_to_module(new_lm_head, hook)\n            old_lm_head_requires_grad = old_lm_head.weight.requires_grad\n            new_lm_head.requires_grad_(old_lm_head_requires_grad)\n            self.set_output_embeddings(new_lm_head)\n\n        return self.get_input_embeddings()\n\n    def _get_resized_embeddings(\n        self,\n        old_embeddings: nn.Embedding,\n        new_num_tokens: Optional[int] = None,\n        pad_to_multiple_of: Optional[int] = None,\n    ) -> nn.Embedding:\n        """\n        Build a resized Embedding Module from a provided token Embedding Module. Increasing the size will add newly\n        initialized vectors at the end. Reducing the size will remove vectors from the end\n\n        Args:\n            old_embeddings (`torch.nn.Embedding`):\n                Old embeddings to be resized.\n            new_num_tokens (`int`, *optional*):\n                New number of tokens in the embedding matrix.\n\n                Increasing the size will add newly initialized vectors at the end. Reducing the size will remove\n                vectors from the end. If not provided or `None`, just returns a pointer to the input tokens\n                `torch.nn.Embedding` module of the model without doing anything.\n            pad_to_multiple_of (`int`, *optional*):\n                If set will pad the embedding matrix to a multiple of the provided value. If `new_num_tokens` is set to\n                `None` will just pad the embedding to a multiple of `pad_to_multiple_of`.\n\n                This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability\n                `>= 7.5` (Volta), or on TPUs which benefit from having sequence lengths be a multiple of 128. For more\n                details about this, or help on choosing the correct value for resizing, refer to this guide:\n                https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc\n\n\n        Return:\n            `torch.nn.Embedding`: Pointer to the resized Embedding Module or the old Embedding Module if\n            `new_num_tokens` is `None`\n        """\n\n        if pad_to_multiple_of is not None:\n            if not isinstance(pad_to_multiple_of, int):\n                raise ValueError(\n                    f"Asking to pad the embedding matrix to a multiple of `{pad_to_multiple_of}`, which is not and integer. Please make sure to pass an integer"\n                )\n            if new_num_tokens is None:\n                new_num_tokens = old_embeddings.weight.shape[0]\n            new_num_tokens = ((new_num_tokens + pad_to_multiple_of - 1) // pad_to_multiple_of) * pad_to_multiple_of\n        else:\n            logger.info(\n                "You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding"\n                f" dimension will be {new_num_tokens}. This might induce some performance reduction as *Tensor Cores* will not be available."\n                " For more details about this, or help on choosing the correct value for resizing, refer to this guide:"\n                " https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc"\n            )\n\n        if new_num_tokens is None:\n            return old_embeddings\n\n        if is_deepspeed_zero3_enabled():\n            import deepspeed\n\n            with deepspeed.zero.GatheredParameters(old_embeddings.weight, modifier_rank=None):\n                old_num_tokens, old_embedding_dim = old_embeddings.weight.size()\n        else:\n            old_num_tokens, old_embedding_dim = old_embeddings.weight.size()\n\n        if old_num_tokens == new_num_tokens and not is_deepspeed_zero3_enabled():\n            return old_embeddings\n\n        if not isinstance(old_embeddings, nn.Embedding):\n            raise TypeError(\n                f"Old embeddings are of type {type(old_embeddings)}, which is not an instance of {nn.Embedding}. You"\n                " should either use a different resize function or make sure that `old_embeddings` are an instance of"\n                f" {nn.Embedding}."\n            )\n\n        # Build new embeddings\n\n        # When using DeepSpeed ZeRO-3, we shouldn\'t create new embeddings with DeepSpeed init\n        # because the shape of the new embedding layer is used across various modeling files\n        # as well as to update config vocab size. Shape will be 0 when using DeepSpeed init leading\n        # to errors when training.\n        new_embeddings = nn.Embedding(\n            new_num_tokens,\n            old_embedding_dim,\n            device=old_embeddings.weight.device,\n            dtype=old_embeddings.weight.dtype,\n        )\n\n        # initialize all new embeddings (in particular added tokens)\n        self._init_weights(new_embeddings)\n\n        # Copy token embeddings from the previous weights\n\n        # numbers of tokens to copy\n        n = min(old_num_tokens, new_num_tokens)\n\n        if is_deepspeed_zero3_enabled():\n            import deepspeed\n\n            params = [old_embeddings.weight, new_embeddings.weight]\n            with deepspeed.zero.GatheredParameters(params, modifier_rank=0):\n                new_embeddings.weight.data[:n, :] = old_embeddings.weight.data[:n, :]\n        else:\n            new_embeddings.weight.data[:n, :] = old_embeddings.weight.data[:n, :]\n\n        return new_embeddings\n\n    def _get_resized_lm_head(\n        self, old_lm_head: nn.Linear, new_num_tokens: Optional[int] = None, transposed: Optional[bool] = False\n    ) -> nn.Linear:\n        """\n        Build a resized Linear Module from a provided old Linear Module. Increasing the size will add newly initialized\n        vectors at the end. Reducing the size will remove vectors from the end\n\n        Args:\n            old_lm_head (`torch.nn.Linear`):\n                Old lm head liner layer to be resized.\n            new_num_tokens (`int`, *optional*):\n                New number of tokens in the linear matrix.\n\n                Increasing the size will add newly initialized vectors at the end. Reducing the size will remove\n                vectors from the end. If not provided or `None`, just returns a pointer to the input tokens\n                `torch.nn.Linear` module of the model without doing anything. transposed (`bool`, *optional*, defaults\n                to `False`): Whether `old_lm_head` is transposed or not. If True `old_lm_head.size()` is `lm_head_dim,\n                vocab_size` else `vocab_size, lm_head_dim`.\n\n        Return:\n            `torch.nn.Linear`: Pointer to the resized Linear Module or the old Linear Module if `new_num_tokens` is\n            `None`\n        """\n        if new_num_tokens is None:\n            return old_lm_head\n\n        if is_deepspeed_zero3_enabled():\n            import deepspeed\n\n            with deepspeed.zero.GatheredParameters(old_lm_head.weight, modifier_rank=None):\n                old_num_tokens, old_lm_head_dim = (\n                    old_lm_head.weight.size() if not transposed else old_lm_head.weight.t().size()\n                )\n        else:\n            old_num_tokens, old_lm_head_dim = (\n                old_lm_head.weight.size() if not transposed else old_lm_head.weight.t().size()\n            )\n\n        if old_num_tokens == new_num_tokens and not is_deepspeed_zero3_enabled():\n            return old_lm_head\n\n        if not isinstance(old_lm_head, nn.Linear):\n            raise TypeError(\n                f"Old language model head is of type {type(old_lm_head)}, which is not an instance of {nn.Linear}. You"\n                " should either use a different resize function or make sure that `old_lm_head` are an instance of"\n                f" {nn.Linear}."\n            )\n\n        # Build new lm head\n        new_lm_head_shape = (old_lm_head_dim, new_num_tokens) if not transposed else (new_num_tokens, old_lm_head_dim)\n        has_new_lm_head_bias = old_lm_head.bias is not None\n\n        # When using DeepSpeed ZeRO-3, we shouldn\'t create new embeddings with DeepSpeed init\n        # because the shape of the new embedding layer is used across various modeling files\n        # as well as to update config vocab size. Shape will be 0 when using DeepSpeed init leading\n        # to errors when training.\n        new_lm_head = nn.Linear(\n            *new_lm_head_shape,\n            bias=has_new_lm_head_bias,\n            device=old_lm_head.weight.device,\n            dtype=old_lm_head.weight.dtype,\n        )\n\n        # initialize new lm head (in particular added tokens)\n        self._init_weights(new_lm_head)\n\n        num_tokens_to_copy = min(old_num_tokens, new_num_tokens)\n\n        if is_deepspeed_zero3_enabled():\n            import deepspeed\n\n            params = [old_lm_head.weight, old_lm_head.bias, new_lm_head.weight, new_lm_head.bias]\n            with deepspeed.zero.GatheredParameters(params, modifier_rank=0):\n                self._copy_lm_head_original_to_resized(\n                    new_lm_head, old_lm_head, num_tokens_to_copy, transposed, has_new_lm_head_bias\n                )\n        else:\n            self._copy_lm_head_original_to_resized(\n                new_lm_head, old_lm_head, num_tokens_to_copy, transposed, has_new_lm_head_bias\n            )\n\n        return new_lm_head\n\n    def _copy_lm_head_original_to_resized(\n        self, new_lm_head, old_lm_head, num_tokens_to_copy, transposed, has_new_lm_head_bias\n    ):\n        # Copy old lm head weights to new lm head\n        if not transposed:\n            new_lm_head.weight.data[:num_tokens_to_copy, :] = old_lm_head.weight.data[:num_tokens_to_copy, :]\n        else:\n            new_lm_head.weight.data[:, :num_tokens_to_copy] = old_lm_head.weight.data[:, :num_tokens_to_copy]\n\n        # Copy bias weights to new lm head\n        if has_new_lm_head_bias:\n            new_lm_head.bias.data[:num_tokens_to_copy] = old_lm_head.bias.data[:num_tokens_to_copy]\n\n    def resize_position_embeddings(self, new_num_position_embeddings: int):\n        raise NotImplementedError(\n            f"`resize_position_embeddings` is not implemented for {self.__class__}`. To implement it, you should "\n            f"overwrite this method in the class {self.__class__} in `modeling_{self.__class__.__module__}.py`"\n        )\n\n    def get_position_embeddings(self) -> Union[nn.Embedding, Tuple[nn.Embedding]]:\n        raise NotImplementedError(\n            f"`get_position_embeddings` is not implemented for {self.__class__}`. To implement it, you should "\n            f"overwrite this method in the class {self.__class__} in `modeling_{self.__class__.__module__}.py`"\n        )\n\n    def init_weights(self):\n        """\n        If needed prunes and maybe initializes weights. If using a custom `PreTrainedModel`, you need to implement any\n        initialization logic in `_init_weights`.\n        """\n        # Prune heads if needed\n        if self.config.pruned_heads:\n            self.prune_heads(self.config.pruned_heads)\n\n        if _init_weights:\n            # Initialize weights\n            self.apply(self._initialize_weights)\n\n            # Tie weights should be skipped when not initializing all weights\n            # since from_pretrained(...) calls tie weights anyways\n            self.tie_weights()\n\n    def prune_heads(self, heads_to_prune: Dict[int, List[int]]):\n        """\n        Prunes heads of the base model.\n\n        Arguments:\n            heads_to_prune (`Dict[int, List[int]]`):\n                Dictionary with keys being selected layer indices (`int`) and associated values being the list of heads\n                to prune in said layer (list of `int`). For instance {1: [0, 2], 2: [2, 3]} will prune heads 0 and 2 on\n                layer 1 and heads 2 and 3 on layer 2.\n        """\n        # save new sets of pruned heads as union of previously stored pruned heads and newly pruned heads\n        for layer, heads in heads_to_prune.items():\n            union_heads = set(self.config.pruned_heads.get(layer, [])) | set(heads)\n            self.config.pruned_heads[layer] = list(union_heads)  # Unfortunately we have to store it as list for JSON\n\n        self.base_model._prune_heads(heads_to_prune)\n\n    def gradient_checkpointing_enable(self, gradient_checkpointing_kwargs=None):\n        """\n        Activates gradient checkpointing for the current model.\n\n        Note that in other frameworks this feature can be referred to as "activation checkpointing" or "checkpoint\n        activations".\n\n        We pass the `__call__` method of the modules instead of `forward` because `__call__` attaches all the hooks of\n        the module. https://discuss.pytorch.org/t/any-different-between-model-input-and-model-forward-input/3690/2\n\n        Args:\n            gradient_checkpointing_kwargs (dict, *optional*):\n                Additional keyword arguments passed along to the `torch.utils.checkpoint.checkpoint` function.\n        """\n        if not self.supports_gradient_checkpointing:\n            raise ValueError(f"{self.__class__.__name__} does not support gradient checkpointing.")\n\n        if gradient_checkpointing_kwargs is None:\n            gradient_checkpointing_kwargs = {}\n\n        gradient_checkpointing_func = functools.partial(checkpoint, **gradient_checkpointing_kwargs)\n\n        # For old GC format (transformers < 4.35.0) for models that live on the Hub\n        # we will fall back to the overwritten `_set_gradient_checkpointing` methid\n        _is_using_old_format = "value" in inspect.signature(self._set_gradient_checkpointing).parameters\n\n        if not _is_using_old_format:\n            self._set_gradient_checkpointing(enable=True, gradient_checkpointing_func=gradient_checkpointing_func)\n        else:\n            self.apply(partial(self._set_gradient_checkpointing, value=True))\n            logger.warn(\n                "You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it)."\n                "Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model."\n            )\n\n        if getattr(self, "_hf_peft_config_loaded", False):\n            # When using PEFT + gradient checkpointing + Trainer we need to make sure the input has requires_grad=True\n            # we do it also on PEFT: https://github.com/huggingface/peft/blob/85013987aa82aa1af3da1236b6902556ce3e483e/src/peft/peft_model.py#L334\n            # When training with PEFT, only LoRA layers will have requires grad set to True, but the output of frozen layers need to propagate\n            # the gradients to make sure the gradient flows.\n            self.enable_input_require_grads()\n\n    def _set_gradient_checkpointing(self, enable: bool = True, gradient_checkpointing_func: Callable = checkpoint):\n        is_gradient_checkpointing_set = False\n\n        # Apply it on the top-level module in case the top-level modules supports it\n        # for example, LongT5Stack inherits from `PreTrainedModel`.\n        if hasattr(self, "gradient_checkpointing"):\n            self._gradient_checkpointing_func = gradient_checkpointing_func\n            self.gradient_checkpointing = enable\n            is_gradient_checkpointing_set = True\n\n        for module in self.modules():\n            if hasattr(module, "gradient_checkpointing"):\n                module._gradient_checkpointing_func = gradient_checkpointing_func\n                module.gradient_checkpointing = enable\n                is_gradient_checkpointing_set = True\n\n        if not is_gradient_checkpointing_set:\n            raise ValueError(\n                f"{self.__class__.__name__} is not compatible with gradient checkpointing. Make sure all the architecture support it by setting a boolean attribute"\n                " `gradient_checkpointing` to modules of the model that uses checkpointing."\n            )\n\n    def gradient_checkpointing_disable(self):\n        """\n        Deactivates gradient checkpointing for the current model.\n\n        Note that in other frameworks this feature can be referred to as "activation checkpointing" or "checkpoint\n        activations".\n        """\n        if self.supports_gradient_checkpointing:\n            # For old GC format (transformers < 4.35.0) for models that live on the Hub\n            # we will fall back to the overwritten `_set_gradient_checkpointing` methid\n            _is_using_old_format = "value" in inspect.signature(self._set_gradient_checkpointing).parameters\n            if not _is_using_old_format:\n                self._set_gradient_checkpointing(enable=False)\n            else:\n                logger.warn(\n                    "You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it)."\n                    "Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model."\n                )\n                self.apply(partial(self._set_gradient_checkpointing, value=False))\n\n        if getattr(self, "_hf_peft_config_loaded", False):\n            self.disable_input_require_grads()\n\n    @property\n    def is_gradient_checkpointing(self) -> bool:\n        """\n        Whether gradient checkpointing is activated for this model or not.\n\n        Note that in other frameworks this feature can be referred to as "activation checkpointing" or "checkpoint\n        activations".\n        """\n        return any(hasattr(m, "gradient_checkpointing") and m.gradient_checkpointing for m in self.modules())\n\n    def save_pretrained(\n        self,\n        save_directory: Union[str, os.PathLike],\n        is_main_process: bool = True,\n        state_dict: Optional[dict] = None,\n        save_function: Callable = torch.save,\n        push_to_hub: bool = False,\n        max_shard_size: Union[int, str] = "5GB",\n        safe_serialization: bool = True,\n        variant: Optional[str] = None,\n        token: Optional[Union[str, bool]] = None,\n        save_peft_format: bool = True,\n        **kwargs,\n    ):\n        """\n        Save a model and its configuration file to a directory, so that it can be re-loaded using the\n        [`~PreTrainedModel.from_pretrained`] class method.\n\n        Arguments:\n            save_directory (`str` or `os.PathLike`):\n                Directory to which to save. Will be created if it doesn\'t exist.\n            is_main_process (`bool`, *optional*, defaults to `True`):\n                Whether the process calling this is the main process or not. Useful when in distributed training like\n                TPUs and need to call this function on all processes. In this case, set `is_main_process=True` only on\n                the main process to avoid race conditions.\n            state_dict (nested dictionary of `torch.Tensor`):\n                The state dictionary of the model to save. Will default to `self.state_dict()`, but can be used to only\n                save parts of the model or if special precautions need to be taken when recovering the state dictionary\n                of a model (like when using model parallelism).\n            save_function (`Callable`):\n                The function to use to save the state dictionary. Useful on distributed training like TPUs when one\n                need to replace `torch.save` by another method.\n            push_to_hub (`bool`, *optional*, defaults to `False`):\n                Whether or not to push your model to the Hugging Face model hub after saving it. You can specify the\n                repository you want to push to with `repo_id` (will default to the name of `save_directory` in your\n                namespace).\n            max_shard_size (`int` or `str`, *optional*, defaults to `"5GB"`):\n                The maximum size for a checkpoint before being sharded. Checkpoints shard will then be each of size\n                lower than this size. If expressed as a string, needs to be digits followed by a unit (like `"5MB"`).\n                We default it to 5GB in order for models to be able to run easily on free-tier google colab instances\n                without CPU OOM issues.\n\n                <Tip warning={true}>\n\n                If a single weight of the model is bigger than `max_shard_size`, it will be in its own checkpoint shard\n                which will be bigger than `max_shard_size`.\n\n                </Tip>\n\n            safe_serialization (`bool`, *optional*, defaults to `True`):\n                Whether to save the model using `safetensors` or the traditional PyTorch way (that uses `pickle`).\n            variant (`str`, *optional*):\n                If specified, weights are saved in the format pytorch_model.<variant>.bin.\n            token (`str` or `bool`, *optional*):\n                The token to use as HTTP bearer authorization for remote files. If `True`, or not specified, will use\n                the token generated when running `huggingface-cli login` (stored in `~/.huggingface`).\n            save_peft_format (`bool`, *optional*, defaults to `True`):\n                For backward compatibility with PEFT library, in case adapter weights are attached to the model, all\n                keys of the state dict of adapters needs to be pre-pended with `base_model.model`. Advanced users can\n                disable this behaviours by setting `save_peft_format` to `False`.\n            kwargs (`Dict[str, Any]`, *optional*):\n                Additional key word arguments passed along to the [`~utils.PushToHubMixin.push_to_hub`] method.\n        """\n        use_auth_token = kwargs.pop("use_auth_token", None)\n\n        if use_auth_token is not None:\n            warnings.warn(\n                "The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.",\n                FutureWarning,\n            )\n            if token is not None:\n                raise ValueError(\n                    "`token` and `use_auth_token` are both specified. Please set only the argument `token`."\n                )\n            token = use_auth_token\n\n        if token is not None:\n            kwargs["token"] = token\n\n        _hf_peft_config_loaded = getattr(self, "_hf_peft_config_loaded", False)\n\n        # Checks if the model has been loaded in 8-bit\n        if (\n            getattr(self, "is_loaded_in_8bit", False)\n            and not getattr(self, "is_8bit_serializable", False)\n            and not _hf_peft_config_loaded\n        ):\n            raise ValueError(\n                "You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected"\n                " behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed."\n            )\n\n        # If the model has adapters attached, you can save the adapters\n        if getattr(self, "is_loaded_in_4bit", False) and not _hf_peft_config_loaded:\n            raise NotImplementedError(\n                "You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported"\n            )\n\n        if "save_config" in kwargs:\n            warnings.warn(\n                "`save_config` is deprecated and will be removed in v5 of Transformers. Use `is_main_process` instead."\n            )\n            is_main_process = kwargs.pop("save_config")\n        if safe_serialization and not is_safetensors_available():\n            raise ImportError("`safe_serialization` requires the `safetensors library: `pip install safetensors`.")\n\n        if os.path.isfile(save_directory):\n            logger.error(f"Provided path ({save_directory}) should be a directory, not a file")\n            return\n\n        os.makedirs(save_directory, exist_ok=True)\n\n        if push_to_hub:\n            commit_message = kwargs.pop("commit_message", None)\n            repo_id = kwargs.pop("repo_id", save_directory.split(os.path.sep)[-1])\n            repo_id = self._create_repo(repo_id, **kwargs)\n            files_timestamps = self._get_files_timestamps(save_directory)\n\n        # Only save the model itself if we are using distributed training\n        model_to_save = unwrap_model(self)\n\n        # save the string version of dtype to the config, e.g. convert torch.float32 => "float32"\n        # we currently don\'t use this setting automatically, but may start to use with v5\n        dtype = get_parameter_dtype(model_to_save)\n        model_to_save.config.torch_dtype = str(dtype).split(".")[1]\n\n        # Attach architecture to the config\n        model_to_save.config.architectures = [model_to_save.__class__.__name__]\n\n        # If we have a custom model, we copy the file defining it in the folder and set the attributes so it can be\n        # loaded from the Hub.\n        if self._auto_class is not None:\n            custom_object_save(self, save_directory, config=self.config)\n\n        # Save the config\n        if is_main_process:\n            if not _hf_peft_config_loaded:\n                model_to_save.config.save_pretrained(save_directory)\n            if self.can_generate():\n                model_to_save.generation_config.save_pretrained(save_directory)\n\n            if _hf_peft_config_loaded:\n                logger.info(\n                    "Detected adapters on the model, saving the model in the PEFT format, only adapter weights will be saved."\n                )\n                state_dict = model_to_save.get_adapter_state_dict()\n\n                if save_peft_format:\n                    logger.info(\n                        "To match the expected format of the PEFT library, all keys of the state dict of adapters will be pre-pended with `base_model.model`."\n                    )\n                    peft_state_dict = {}\n                    for key, value in state_dict.items():\n                        peft_state_dict[f"base_model.model.{key}"] = value\n                    state_dict = peft_state_dict\n\n                active_adapter = self.active_adapters()\n\n                if len(active_adapter) > 1:\n                    raise ValueError(\n                        "Multiple active adapters detected, saving multiple active adapters is not supported yet. You can save adapters separately one by one "\n                        "by iteratively calling `model.set_adapter(adapter_name)` then `model.save_pretrained(...)`"\n                    )\n                active_adapter = active_adapter[0]\n\n                current_peft_config = self.peft_config[active_adapter]\n                current_peft_config.save_pretrained(save_directory)\n\n        # Save the model\n        if state_dict is None:\n            state_dict = model_to_save.state_dict()\n\n        # Translate state_dict from smp to hf if saving with smp >= 1.10\n        if IS_SAGEMAKER_MP_POST_1_10:\n            for smp_to_hf, _ in smp.state.module_manager.translate_functions:\n                state_dict = smp_to_hf(state_dict)\n\n        # Handle the case where some state_dict keys shouldn\'t be saved\n        if self._keys_to_ignore_on_save is not None:\n            for ignore_key in self._keys_to_ignore_on_save:\n                if ignore_key in state_dict.keys():\n                    del state_dict[ignore_key]\n        if safe_serialization:\n            # Safetensors does not allow tensor aliasing.\n            # We\'re going to remove aliases before saving\n            ptrs = collections.defaultdict(list)\n            for name, tensor in state_dict.items():\n                # Sometimes in the state_dict we have non-tensor objects.\n                # e.g. in bitsandbytes we have some `str` objects in the state_dict\n                if isinstance(tensor, torch.Tensor):\n                    ptrs[id_tensor_storage(tensor)].append(name)\n                else:\n                    # In the non-tensor case, fall back to the pointer of the object itself\n                    ptrs[id(tensor)].append(name)\n\n            # These are all the pointers of shared tensors.\n            shared_ptrs = {ptr: names for ptr, names in ptrs.items() if len(names) > 1}\n            warn_names = set()\n            for names in shared_ptrs.values():\n                # Removing the keys which are declared as known duplicates on\n                # load. This allows to make sure the name which is kept is consistent.\n                if self._tied_weights_keys is not None:\n                    found = 0\n                    for name in sorted(names):\n                        matches_pattern = any(re.search(pat, name) for pat in self._tied_weights_keys)\n                        if matches_pattern and name in state_dict:\n                            found += 1\n                            if found < len(names):\n                                del state_dict[name]\n\n                # When not all duplicates have been cleaned, still remove those keys, but put a clear warning.\n                # If the link between tensors was done at runtime then `from_pretrained` will not get\n                # the key back leading to random tensor. A proper warning will be shown\n                # during reload (if applicable), but since the file is not necessarily compatible with\n                # the config, better show a proper warning.\n                found = 0\n                for name in names:\n                    if name in state_dict:\n                        found += 1\n                        if found > 1:\n                            del state_dict[name]\n                            warn_names.add(name)\n            if len(warn_names) > 0:\n                logger.warning_once(\n                    f"Removed shared tensor {warn_names} while saving. This should be OK, but check by verifying that you don\'t receive any warning while reloading",\n                )\n\n        # Shard the model if it is too big.\n        if not _hf_peft_config_loaded:\n            weights_name = SAFE_WEIGHTS_NAME if safe_serialization else WEIGHTS_NAME\n            weights_name = _add_variant(weights_name, variant)\n        else:\n            weights_name = ADAPTER_SAFE_WEIGHTS_NAME if safe_serialization else ADAPTER_WEIGHTS_NAME\n\n        shards, index = shard_checkpoint(state_dict, max_shard_size=max_shard_size, weights_name=weights_name)\n\n        # Clean the folder from a previous save\n        for filename in os.listdir(save_directory):\n            full_filename = os.path.join(save_directory, filename)\n            # If we have a shard file that is not going to be replaced, we delete it, but only from the main process\n            # in distributed settings to avoid race conditions.\n            weights_no_suffix = weights_name.replace(".bin", "").replace(".safetensors", "")\n\n            # make sure that file to be deleted matches format of sharded file, e.g. pytorch_model-00001-of-00005\n            filename_no_suffix = filename.replace(".bin", "").replace(".safetensors", "")\n            reg = re.compile(r"(.*?)-\\d{5}-of-\\d{5}")\n\n            if (\n                filename.startswith(weights_no_suffix)\n                and os.path.isfile(full_filename)\n                and filename not in shards.keys()\n                and is_main_process\n                and reg.fullmatch(filename_no_suffix) is not None\n            ):\n                os.remove(full_filename)\n\n        # Save the model\n        for shard_file, shard in shards.items():\n            if safe_serialization:\n                # At some point we will need to deal better with save_function (used for TPU and other distributed\n                # joyfulness), but for now this enough.\n                safe_save_file(shard, os.path.join(save_directory, shard_file), metadata={"format": "pt"})\n            else:\n                save_function(shard, os.path.join(save_directory, shard_file))\n\n        if index is None:\n            path_to_weights = os.path.join(save_directory, _add_variant(WEIGHTS_NAME, variant))\n            logger.info(f"Model weights saved in {path_to_weights}")\n        else:\n            save_index_file = SAFE_WEIGHTS_INDEX_NAME if safe_serialization else WEIGHTS_INDEX_NAME\n            save_index_file = os.path.join(save_directory, _add_variant(save_index_file, variant))\n            # Save the index as well\n            with open(save_index_file, "w", encoding="utf-8") as f:\n                content = json.dumps(index, indent=2, sort_keys=True) + "\\n"\n                f.write(content)\n            logger.info(\n                f"The model is bigger than the maximum size per checkpoint ({max_shard_size}) and is going to be "\n                f"split in {len(shards)} checkpoint shards. You can find where each parameters has been saved in the "\n                f"index located at {save_index_file}."\n            )\n\n        if push_to_hub:\n            self._upload_modified_files(\n                save_directory,\n                repo_id,\n                files_timestamps,\n                commit_message=commit_message,\n                token=token,\n            )\n\n    def get_memory_footprint(self, return_buffers=True):\n        r"""\n        Get the memory footprint of a model. This will return the memory footprint of the current model in bytes.\n        Useful to benchmark the memory footprint of the current model and design some tests. Solution inspired from the\n        PyTorch discussions: https://discuss.pytorch.org/t/gpu-memory-that-model-uses/56822/2\n\n        Arguments:\n            return_buffers (`bool`, *optional*, defaults to `True`):\n                Whether to return the size of the buffer tensors in the computation of the memory footprint. Buffers\n                are tensors that do not require gradients and not registered as parameters. E.g. mean and std in batch\n                norm layers. Please see: https://discuss.pytorch.org/t/what-pytorch-means-by-buffers/120266/2\n        """\n        mem = sum([param.nelement() * param.element_size() for param in self.parameters()])\n        if return_buffers:\n            mem_bufs = sum([buf.nelement() * buf.element_size() for buf in self.buffers()])\n            mem = mem + mem_bufs\n        return mem\n\n    @wraps(torch.nn.Module.cuda)\n    def cuda(self, *args, **kwargs):\n        # Checks if the model has been loaded in 8-bit\n        if getattr(self, "quantization_method", None) == QuantizationMethod.BITS_AND_BYTES:\n            raise ValueError(\n                "Calling `cuda()` is not supported for `4-bit` or `8-bit` quantized models. Please use the model as it is, since the"\n                " model has already been set to the correct devices and casted to the correct `dtype`."\n            )\n        else:\n            return super().cuda(*args, **kwargs)\n\n    @wraps(torch.nn.Module.to)\n    def to(self, *args, **kwargs):\n        # Checks if the model has been loaded in 8-bit\n        if getattr(self, "quantization_method", None) == QuantizationMethod.BITS_AND_BYTES:\n            raise ValueError(\n                "`.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the"\n                " model has already been set to the correct devices and casted to the correct `dtype`."\n            )\n        elif getattr(self, "quantization_method", None) == QuantizationMethod.GPTQ:\n            # For GPTQ models, we prevent users from casting the model to another dytpe to restrict unwanted behaviours.\n            # the correct API should be to load the model with the desired dtype directly through `from_pretrained`.\n            dtype_present_in_args = False\n\n            if "dtype" not in kwargs:\n                for arg in args:\n                    if isinstance(arg, torch.dtype):\n                        dtype_present_in_args = True\n                        break\n            else:\n                dtype_present_in_args = True\n\n            if dtype_present_in_args:\n                raise ValueError(\n                    "You cannot cast a GPTQ model in a new `dtype`. Make sure to load the model using `from_pretrained` using the desired"\n                    " `dtype` by passing the correct `torch_dtype` argument."\n                )\n        return super().to(*args, **kwargs)\n\n    def half(self, *args):\n        # Checks if the model is quantized\n        if getattr(self, "is_quantized", False):\n            raise ValueError(\n                "`.half()` is not supported for quantized model. Please use the model as it is, since the"\n                " model has already been casted to the correct `dtype`."\n            )\n        else:\n            return super().half(*args)\n\n    def float(self, *args):\n        # Checks if the model is quantized\n        if getattr(self, "is_quantized", False):\n            raise ValueError(\n                "`.float()` is not supported for quantized model. Please use the model as it is, since the"\n                " model has already been casted to the correct `dtype`."\n            )\n        else:\n            return super().float(*args)\n\n    @classmethod\n    def from_pretrained(\n        cls,\n        pretrained_model_name_or_path: Optional[Union[str, os.PathLike]],\n        *model_args,\n        config: Optional[Union[PretrainedConfig, str, os.PathLike]] = None,\n        cache_dir: Optional[Union[str, os.PathLike]] = None,\n        ignore_mismatched_sizes: bool = False,\n        force_download: bool = False,\n        local_files_only: bool = False,\n        token: Optional[Union[str, bool]] = None,\n        revision: str = "main",\n        use_safetensors: bool = None,\n        **kwargs,\n    ):\n        r"""\n        Instantiate a pretrained pytorch model from a pre-trained model configuration.\n\n        The model is set in evaluation mode by default using `model.eval()` (Dropout modules are deactivated). To train\n        the model, you should first set it back in training mode with `model.train()`.\n\n        The warning *Weights from XXX not initialized from pretrained model* means that the weights of XXX do not come\n        pretrained with the rest of the model. It is up to you to train those weights with a downstream fine-tuning\n        task.\n\n        The warning *Weights from XXX not used in YYY* means that the layer XXX is not used by YYY, therefore those\n        weights are discarded.\n\n        Parameters:\n            pretrained_model_name_or_path (`str` or `os.PathLike`, *optional*):\n                Can be either:\n\n                    - A string, the *model id* of a pretrained model hosted inside a model repo on huggingface.co.\n                      Valid model ids can be located at the root-level, like `bert-base-uncased`, or namespaced under a\n                      user or organization name, like `dbmdz/bert-base-german-cased`.\n                    - A path to a *directory* containing model weights saved using\n                      [`~PreTrainedModel.save_pretrained`], e.g., `./my_model_directory/`.\n                    - A path or url to a *tensorflow index checkpoint file* (e.g, `./tf_model/model.ckpt.index`). In\n                      this case, `from_tf` should be set to `True` and a configuration object should be provided as\n                      `config` argument. This loading path is slower than converting the TensorFlow checkpoint in a\n                      PyTorch model using the provided conversion scripts and loading the PyTorch model afterwards.\n                    - A path or url to a model folder containing a *flax checkpoint file* in *.msgpack* format (e.g,\n                      `./flax_model/` containing `flax_model.msgpack`). In this case, `from_flax` should be set to\n                      `True`.\n                    - `None` if you are both providing the configuration and state dictionary (resp. with keyword\n                      arguments `config` and `state_dict`).\n            model_args (sequence of positional arguments, *optional*):\n                All remaining positional arguments will be passed to the underlying model\'s `__init__` method.\n            config (`Union[PretrainedConfig, str, os.PathLike]`, *optional*):\n                Can be either:\n\n                    - an instance of a class derived from [`PretrainedConfig`],\n                    - a string or path valid as input to [`~PretrainedConfig.from_pretrained`].\n\n                Configuration for the model to use instead of an automatically loaded configuration. Configuration can\n                be automatically loaded when:\n\n                    - The model is a model provided by the library (loaded with the *model id* string of a pretrained\n                      model).\n                    - The model was saved using [`~PreTrainedModel.save_pretrained`] and is reloaded by supplying the\n                      save directory.\n                    - The model is loaded by supplying a local directory as `pretrained_model_name_or_path` and a\n                      configuration JSON file named *config.json* is found in the directory.\n            state_dict (`Dict[str, torch.Tensor]`, *optional*):\n                A state dictionary to use instead of a state dictionary loaded from saved weights file.\n\n                This option can be used if you want to create a model from a pretrained configuration but load your own\n                weights. In this case though, you should check if using [`~PreTrainedModel.save_pretrained`] and\n                [`~PreTrainedModel.from_pretrained`] is not a simpler option.\n            cache_dir (`Union[str, os.PathLike]`, *optional*):\n                Path to a directory in which a downloaded pretrained model configuration should be cached if the\n                standard cache should not be used.\n            from_tf (`bool`, *optional*, defaults to `False`):\n                Load the model weights from a TensorFlow checkpoint save file (see docstring of\n                `pretrained_model_name_or_path` argument).\n            from_flax (`bool`, *optional*, defaults to `False`):\n                Load the model weights from a Flax checkpoint save file (see docstring of\n                `pretrained_model_name_or_path` argument).\n            ignore_mismatched_sizes (`bool`, *optional*, defaults to `False`):\n                Whether or not to raise an error if some of the weights from the checkpoint do not have the same size\n                as the weights of the model (if for instance, you are instantiating a model with 10 labels from a\n                checkpoint with 3 labels).\n            force_download (`bool`, *optional*, defaults to `False`):\n                Whether or not to force the (re-)download of the model weights and configuration files, overriding the\n                cached versions if they exist.\n            resume_download (`bool`, *optional*, defaults to `False`):\n                Whether or not to delete incompletely received files. Will attempt to resume the download if such a\n                file exists.\n            proxies (`Dict[str, str]`, *optional*):\n                A dictionary of proxy servers to use by protocol or endpoint, e.g., `{\'http\': \'foo.bar:3128\',\n                \'http://hostname\': \'foo.bar:4012\'}`. The proxies are used on each request.\n            output_loading_info(`bool`, *optional*, defaults to `False`):\n                Whether ot not to also return a dictionary containing missing keys, unexpected keys and error messages.\n            local_files_only(`bool`, *optional*, defaults to `False`):\n                Whether or not to only look at local files (i.e., do not try to download the model).\n            token (`str` or `bool`, *optional*):\n                The token to use as HTTP bearer authorization for remote files. If `True`, or not specified, will use\n                the token generated when running `huggingface-cli login` (stored in `~/.huggingface`).\n            revision (`str`, *optional*, defaults to `"main"`):\n                The specific model version to use. It can be a branch name, a tag name, or a commit id, since we use a\n                git-based system for storing models and other artifacts on huggingface.co, so `revision` can be any\n                identifier allowed by git.\n\n                <Tip>\n\n                To test a pull request you made on the Hub, you can pass `revision="refs/pr/<pr_number>".\n\n                </Tip>\n\n            mirror (`str`, *optional*):\n                Mirror source to accelerate downloads in China. If you are from China and have an accessibility\n                problem, you can set this option to resolve it. Note that we do not guarantee the timeliness or safety.\n                Please refer to the mirror site for more information.\n            _fast_init(`bool`, *optional*, defaults to `True`):\n                Whether or not to disable fast initialization.\n\n                <Tip warning={true}>\n\n                One should only disable *_fast_init* to ensure backwards compatibility with `transformers.__version__ <\n                4.6.0` for seeded model initialization. This argument will be removed at the next major version. See\n                [pull request 11471](https://github.com/huggingface/transformers/pull/11471) for more information.\n\n                </Tip>\n\n            > Parameters for big model inference\n\n            low_cpu_mem_usage(`bool`, *optional*):\n                Tries to not use more than 1x model size in CPU memory (including peak memory) while loading the model.\n                This is an experimental feature and a subject to change at any moment.\n            torch_dtype (`str` or `torch.dtype`, *optional*):\n                Override the default `torch.dtype` and load the model under a specific `dtype`. The different options\n                are:\n\n                1. `torch.float16` or `torch.bfloat16` or `torch.float`: load in a specified\n                  `dtype`, ignoring the model\'s `config.torch_dtype` if one exists. If not specified\n                  - the model will get loaded in `torch.float` (fp32).\n\n                2. `"auto"` - A `torch_dtype` entry in the `config.json` file of the model will be\n                  attempted to be used. If this entry isn\'t found then next check the `dtype` of the first weight in\n                  the checkpoint that\'s of a floating point type and use that as `dtype`. This will load the model\n                  using the `dtype` it was saved in at the end of the training. It can\'t be used as an indicator of how\n                  the model was trained. Since it could be trained in one of half precision dtypes, but saved in fp32.\n\n                <Tip>\n\n                For some models the `dtype` they were trained in is unknown - you may try to check the model\'s paper or\n                reach out to the authors and ask them to add this information to the model\'s card and to insert the\n                `torch_dtype` entry in `config.json` on the hub.\n\n                </Tip>\n\n            device_map (`str` or `Dict[str, Union[int, str, torch.device]]` or `int` or `torch.device`, *optional*):\n                A map that specifies where each submodule should go. It doesn\'t need to be refined to each\n                parameter/buffer name, once a given module name is inside, every submodule of it will be sent to the\n                same device. If we only pass the device (*e.g.*, `"cpu"`, `"cuda:1"`, `"mps"`, or a GPU ordinal rank\n                like `1`) on which the model will be allocated, the device map will map the entire model to this\n                device. Passing `device_map = 0` means put the whole model on GPU 0.\n\n                To have Accelerate compute the most optimized `device_map` automatically, set `device_map="auto"`. For\n                more information about each option see [designing a device\n                map](https://hf.co/docs/accelerate/main/en/usage_guides/big_modeling#designing-a-device-map).\n            max_memory (`Dict`, *optional*):\n                A dictionary device identifier to maximum memory. Will default to the maximum memory available for each\n                GPU and the available CPU RAM if unset.\n            offload_folder (`str` or `os.PathLike`, *optional*):\n                If the `device_map` contains any value `"disk"`, the folder where we will offload weights.\n            offload_state_dict (`bool`, *optional*):\n                If `True`, will temporarily offload the CPU state dict to the hard drive to avoid getting out of CPU\n                RAM if the weight of the CPU state dict + the biggest shard of the checkpoint does not fit. Defaults to\n                `True` when there is some disk offload.\n            load_in_8bit (`bool`, *optional*, defaults to `False`):\n                If `True`, will convert the loaded model into mixed-8bit quantized model. To use this feature please\n                install `bitsandbytes` (`pip install -U bitsandbytes`).\n            load_in_4bit (`bool`, *optional*, defaults to `False`):\n                If `True`, will convert the loaded model into 4bit precision quantized model. To use this feature\n                install the latest version of `bitsandbytes` (`pip install -U bitsandbytes`).\n            quantization_config (`Union[QuantizationConfigMixin,Dict]`, *optional*):\n                A dictionary of configuration parameters or a QuantizationConfigMixin object for quantization (e.g\n                bitsandbytes, gptq)\n            subfolder (`str`, *optional*, defaults to `""`):\n                In case the relevant files are located inside a subfolder of the model repo on huggingface.co, you can\n                specify the folder name here.\n            variant (`str`, *optional*):\n                If specified load weights from `variant` filename, *e.g.* pytorch_model.<variant>.bin. `variant` is\n                ignored when using `from_tf` or `from_flax`.\n            use_safetensors (`bool`, *optional*, defaults to `None`):\n                Whether or not to use `safetensors` checkpoints. Defaults to `None`. If not specified and `safetensors`\n                is not installed, it will be set to `False`.\n\n            kwargs (remaining dictionary of keyword arguments, *optional*):\n                Can be used to update the configuration object (after it being loaded) and initiate the model (e.g.,\n                `output_attentions=True`). Behaves differently depending on whether a `config` is provided or\n                automatically loaded:\n\n                    - If a configuration is provided with `config`, `**kwargs` will be directly passed to the\n                      underlying model\'s `__init__` method (we assume all relevant updates to the configuration have\n                      already been done)\n                    - If a configuration is not provided, `kwargs` will be first passed to the configuration class\n                      initialization function ([`~PretrainedConfig.from_pretrained`]). Each key of `kwargs` that\n                      corresponds to a configuration attribute will be used to override said attribute with the\n                      supplied `kwargs` value. Remaining keys that do not correspond to any configuration attribute\n                      will be passed to the underlying model\'s `__init__` function.\n\n        <Tip>\n\n        Activate the special ["offline-mode"](https://huggingface.co/transformers/installation.html#offline-mode) to\n        use this method in a firewalled environment.\n\n        </Tip>\n\n        Examples:\n\n        ```python\n        >>> from transformers import BertConfig, BertModel\n\n        >>> # Download model and configuration from huggingface.co and cache.\n        >>> model = BertModel.from_pretrained("bert-base-uncased")\n        >>> # Model was saved using *save_pretrained(\'./test/saved_model/\')* (for example purposes, not runnable).\n        >>> model = BertModel.from_pretrained("./test/saved_model/")\n        >>> # Update configuration during loading.\n        >>> model = BertModel.from_pretrained("bert-base-uncased", output_attentions=True)\n        >>> assert model.config.output_attentions == True\n        >>> # Loading from a TF checkpoint file instead of a PyTorch model (slower, for example purposes, not runnable).\n        >>> config = BertConfig.from_json_file("./tf_model/my_tf_model_config.json")\n        >>> model = BertModel.from_pretrained("./tf_model/my_tf_checkpoint.ckpt.index", from_tf=True, config=config)\n        >>> # Loading from a Flax checkpoint file instead of a PyTorch model (slower)\n        >>> model = BertModel.from_pretrained("bert-base-uncased", from_flax=True)\n        ```\n\n        * `low_cpu_mem_usage` algorithm:\n\n        This is an experimental function that loads the model using ~1x model size CPU memory\n\n        Here is how it works:\n\n        1. save which state_dict keys we have\n        2. drop state_dict before the model is created, since the latter takes 1x model size CPU memory\n        3. after the model has been instantiated switch to the meta device all params/buffers that\n        are going to be replaced from the loaded state_dict\n        4. load state_dict 2nd time\n        5. replace the params/buffers from the state_dict\n\n        Currently, it can\'t handle deepspeed ZeRO stage 3 and ignores loading errors\n\n        """\n        state_dict = kwargs.pop("state_dict", None)\n        from_tf = kwargs.pop("from_tf", False)\n        from_flax = kwargs.pop("from_flax", False)\n        resume_download = kwargs.pop("resume_download", False)\n        proxies = kwargs.pop("proxies", None)\n        output_loading_info = kwargs.pop("output_loading_info", False)\n        use_auth_token = kwargs.pop("use_auth_token", None)\n        trust_remote_code = kwargs.pop("trust_remote_code", None)\n        _ = kwargs.pop("mirror", None)\n        from_pipeline = kwargs.pop("_from_pipeline", None)\n        from_auto_class = kwargs.pop("_from_auto", False)\n        _fast_init = kwargs.pop("_fast_init", True)\n        torch_dtype = kwargs.pop("torch_dtype", None)\n        low_cpu_mem_usage = kwargs.pop("low_cpu_mem_usage", None)\n        device_map = kwargs.pop("device_map", None)\n        max_memory = kwargs.pop("max_memory", None)\n        offload_folder = kwargs.pop("offload_folder", None)\n        offload_state_dict = kwargs.pop("offload_state_dict", False)\n        load_in_8bit = kwargs.pop("load_in_8bit", False)\n        load_in_4bit = kwargs.pop("load_in_4bit", False)\n        quantization_config = kwargs.pop("quantization_config", None)\n        subfolder = kwargs.pop("subfolder", "")\n        commit_hash = kwargs.pop("_commit_hash", None)\n        variant = kwargs.pop("variant", None)\n        adapter_kwargs = kwargs.pop("adapter_kwargs", {})\n        adapter_name = kwargs.pop("adapter_name", "default")\n        use_flash_attention_2 = kwargs.pop("use_flash_attention_2", False)\n\n        if is_fsdp_enabled():\n            low_cpu_mem_usage = True\n\n        if use_auth_token is not None:\n            warnings.warn(\n                "The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.",\n                FutureWarning,\n            )\n            if token is not None:\n                raise ValueError(\n                    "`token` and `use_auth_token` are both specified. Please set only the argument `token`."\n                )\n            token = use_auth_token\n\n        if token is not None and adapter_kwargs is not None and "token" not in adapter_kwargs:\n            adapter_kwargs["token"] = token\n\n        if use_safetensors is None and not is_safetensors_available():\n            use_safetensors = False\n\n        if is_bitsandbytes_available():\n            is_8bit_serializable = version.parse(importlib.metadata.version("bitsandbytes")) > version.parse("0.37.2")\n        else:\n            is_8bit_serializable = False\n\n        if trust_remote_code is True:\n            logger.warning(\n                "The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is"\n                " ignored."\n            )\n\n        if commit_hash is None:\n            if not isinstance(config, PretrainedConfig):\n                # We make a call to the config file first (which may be absent) to get the commit hash as soon as possible\n                resolved_config_file = cached_file(\n                    pretrained_model_name_or_path,\n                    CONFIG_NAME,\n                    cache_dir=cache_dir,\n                    force_download=force_download,\n                    resume_download=resume_download,\n                    proxies=proxies,\n                    local_files_only=local_files_only,\n                    token=token,\n                    revision=revision,\n                    subfolder=subfolder,\n                    _raise_exceptions_for_missing_entries=False,\n                    _raise_exceptions_for_connection_errors=False,\n                )\n                commit_hash = extract_commit_hash(resolved_config_file, commit_hash)\n            else:\n                commit_hash = getattr(config, "_commit_hash", None)\n\n        if is_peft_available():\n            _adapter_model_path = adapter_kwargs.pop("_adapter_model_path", None)\n\n            if _adapter_model_path is None:\n                _adapter_model_path = find_adapter_config_file(\n                    pretrained_model_name_or_path,\n                    cache_dir=cache_dir,\n                    force_download=force_download,\n                    resume_download=resume_download,\n                    proxies=proxies,\n                    local_files_only=local_files_only,\n                    _commit_hash=commit_hash,\n                    **adapter_kwargs,\n                )\n            if _adapter_model_path is not None and os.path.isfile(_adapter_model_path):\n                with open(_adapter_model_path, "r", encoding="utf-8") as f:\n                    _adapter_model_path = pretrained_model_name_or_path\n                    pretrained_model_name_or_path = json.load(f)["base_model_name_or_path"]\n        else:\n            _adapter_model_path = None\n\n        # change device_map into a map if we passed an int, a str or a torch.device\n        if isinstance(device_map, torch.device):\n            device_map = {"": device_map}\n        elif isinstance(device_map, str) and device_map not in ["auto", "balanced", "balanced_low_0", "sequential"]:\n            try:\n                device_map = {"": torch.device(device_map)}\n            except RuntimeError:\n                raise ValueError(\n                    "When passing device_map as a string, the value needs to be a device name (e.g. cpu, cuda:0) or "\n                    f"\'auto\', \'balanced\', \'balanced_low_0\', \'sequential\' but found {device_map}."\n                )\n        elif isinstance(device_map, int):\n            if device_map < 0:\n                raise ValueError(\n                    "You can\'t pass device_map as a negative int. If you want to put the model on the cpu, pass device_map = \'cpu\' "\n                )\n            else:\n                device_map = {"": device_map}\n\n        if device_map is not None:\n            if low_cpu_mem_usage is None:\n                low_cpu_mem_usage = True\n            elif not low_cpu_mem_usage:\n                raise ValueError("Passing along a `device_map` requires `low_cpu_mem_usage=True`")\n\n        if low_cpu_mem_usage:\n            if device_map is not None:\n                # The max memory utils require PyTorch >= 1.10 to have torch.cuda.mem_get_info.\n                require_version_core("torch>=1.10")\n\n            if is_deepspeed_zero3_enabled():\n                raise ValueError(\n                    "DeepSpeed Zero-3 is not compatible with `low_cpu_mem_usage=True` or with passing a `device_map`."\n                )\n            elif not is_accelerate_available():\n                raise ImportError(\n                    "Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`"\n                )\n\n        quantization_method_from_args = None\n        if quantization_config is not None:\n            quantization_method_from_args = getattr(\n                quantization_config, "quant_method", QuantizationMethod.BITS_AND_BYTES\n            )\n\n            if quantization_method_from_args == QuantizationMethod.AWQ:\n                raise ValueError(\n                    "You cannot pass an `AwqConfig` when loading a model as you can only use AWQ models"\n                    " for inference. To quantize transformers models with AWQ algorithm, please refer to our"\n                    " quantization docs: https://huggingface.co/docs/transformers/main_classes/quantization "\n                )\n\n        if quantization_config is None and (load_in_8bit or load_in_4bit):\n            quantization_method_from_args = QuantizationMethod.BITS_AND_BYTES\n            quantization_config, kwargs = BitsAndBytesConfig.from_dict(\n                config_dict={"load_in_8bit": load_in_8bit, "load_in_4bit": load_in_4bit},\n                return_unused_kwargs=True,\n                **kwargs,\n            )\n        elif quantization_method_from_args == QuantizationMethod.BITS_AND_BYTES:\n            load_in_8bit = quantization_config.load_in_8bit\n            load_in_4bit = quantization_config.load_in_4bit\n\n            quantization_config_kwargs = {\n                k: v for k, v in kwargs.items() if k in inspect.signature(BitsAndBytesConfig).parameters\n            }\n\n            if len(quantization_config_kwargs) > 0:\n                raise ValueError(\n                    "You can\'t pass `load_in_8bit` or any other `BitsAndBytesConfig` argument as a kwarg when passing "\n                    "`quantization_config` argument at the same time."\n                )\n\n        if load_in_8bit or load_in_4bit:\n            if not (is_accelerate_available() and is_bitsandbytes_available()):\n                raise ImportError(\n                    "Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of"\n                    " bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or"\n                    " pip install bitsandbytes` "\n                )\n\n            if torch_dtype is None:\n                # We force the `dtype` to be float16, this is a requirement from `bitsandbytes`\n                logger.info(\n                    f"Overriding torch_dtype={torch_dtype} with `torch_dtype=torch.float16` due to "\n                    "requirements of `bitsandbytes` to enable model loading in 8-bit or 4-bit. "\n                    "Pass your own torch_dtype to specify the dtype of the remaining non-linear layers or pass"\n                    " torch_dtype=torch.float16 to remove this warning."\n                )\n                torch_dtype = torch.float16\n\n            if device_map is None:\n                if torch.cuda.is_available():\n                    device_map = {"": torch.cuda.current_device()}\n                else:\n                    raise RuntimeError("No GPU found. A GPU is needed for quantization.")\n                logger.info(\n                    "The device_map was not initialized. "\n                    "Setting device_map to {\'\':torch.cuda.current_device()}. "\n                    "If you want to use the model for inference, please set device_map =\'auto\' "\n                )\n                if low_cpu_mem_usage is None:\n                    low_cpu_mem_usage = True\n\n            if from_tf or from_flax:\n                raise ValueError(\n                    "Converting into 4-bit or 8-bit weights from tf/flax weights is currently not supported, please make"\n                    " sure the weights are in PyTorch format."\n                )\n\n        user_agent = {"file_type": "model", "framework": "pytorch", "from_auto_class": from_auto_class}\n        if from_pipeline is not None:\n            user_agent["using_pipeline"] = from_pipeline\n\n        if is_offline_mode() and not local_files_only:\n            logger.info("Offline mode: forcing local_files_only=True")\n            local_files_only = True\n\n        # Load config if we don\'t provide a configuration\n        if not isinstance(config, PretrainedConfig):\n            config_path = config if config is not None else pretrained_model_name_or_path\n            config, model_kwargs = cls.config_class.from_pretrained(\n                config_path,\n                cache_dir=cache_dir,\n                return_unused_kwargs=True,\n                force_download=force_download,\n                resume_download=resume_download,\n                proxies=proxies,\n                local_files_only=local_files_only,\n                token=token,\n                revision=revision,\n                subfolder=subfolder,\n                _from_auto=from_auto_class,\n                _from_pipeline=from_pipeline,\n                **kwargs,\n            )\n        else:\n            model_kwargs = kwargs\n\n        quantizer = None\n        quantization_method_from_config = None\n        if hasattr(config, "quantization_config"):\n            quantization_method_from_config = config.quantization_config.get(\n                "quant_method", QuantizationMethod.BITS_AND_BYTES\n            )\n        if quantization_method_from_config is not None and quantization_method_from_args is not None:\n            if quantization_method_from_config != quantization_method_from_args:\n                raise ValueError(\n                    f"The model is already quantized with {quantization_method_from_config}. "\n                    f"You can\'t quantize it again with {quantization_method_from_args}"\n                )\n        if quantization_method_from_config == QuantizationMethod.GPTQ and quantization_method_from_args is not None:\n            loading_attr_dict = quantization_config.get_loading_attributes()\n            for attr, val in loading_attr_dict.items():\n                config.quantization_config[attr] = val\n            quantization_method_from_args = None\n            logger.warning(\n                "You passed `quantization_config` to `from_pretrained` but the model you\'re loading already has a "\n                "`quantization_config` attribute and has already quantized weights. However, loading attributes"\n                " (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored."\n            )\n        if (\n            quantization_method_from_args == QuantizationMethod.GPTQ\n            or quantization_method_from_config == QuantizationMethod.GPTQ\n        ):\n            gptq_supports_cpu = version.parse(importlib.metadata.version("auto-gptq")) > version.parse("0.4.2")\n            if not gptq_supports_cpu and not torch.cuda.is_available():\n                raise RuntimeError("GPU is required to quantize or run quantize model.")\n            elif not (is_optimum_available() and is_auto_gptq_available()):\n                raise ImportError(\n                    "Loading a GPTQ quantized model requires optimum (`pip install optimum`) and auto-gptq library (`pip install auto-gptq`)"\n                )\n            elif version.parse(importlib.metadata.version("auto_gptq")) < version.parse("0.4.2"):\n                raise ImportError(\n                    "You need a version of auto_gptq >= 0.4.2 to use GPTQ: `pip install --upgrade auto-gptq`"\n                )\n            else:\n                # Need to protect the import\n                from optimum.gptq import GPTQQuantizer\n            if quantization_method_from_config == QuantizationMethod.GPTQ:\n                quantization_config = GPTQConfig.from_dict(config.quantization_config)\n                config.quantization_config = quantization_config\n            if torch_dtype is None:\n                torch_dtype = torch.float16\n            else:\n                logger.info("We suggest you to set `torch_dtype=torch.float16` for better efficiency with GPTQ.")\n            quantizer = GPTQQuantizer.from_dict(quantization_config.to_dict_optimum())\n        elif quantization_method_from_config == QuantizationMethod.AWQ:\n            if not torch.cuda.is_available():\n                raise RuntimeError("GPU is required to run AWQ quantized model.")\n\n            if not is_auto_awq_available():\n                raise ImportError("Loading an AWQ quantized model requires auto-awq library (`pip install autoawq`)")\n\n            if not is_accelerate_available():\n                raise ImportError("Loading an AWQ quantized model requires accelerate (`pip install accelerate`)")\n\n            if device_map is None:\n                logger.warning(\n                    "You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set "\n                    "your model on a GPU device in order to run your model."\n                )\n            elif device_map is not None:\n                if isinstance(device_map, dict) and ("cpu" in device_map.values() or "disk" in device_map.values()):\n                    raise ValueError(\n                        "You are attempting to load an AWQ model with a device_map that contains a CPU or disk device."\n                        " This is not supported. Please remove the CPU or disk device from the device_map."\n                    )\n\n            if torch_dtype is None:\n                torch_dtype = torch.float16\n            else:\n                logger.info("We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.")\n\n            # Force-set to `True` for more mem efficiency\n            if low_cpu_mem_usage is None:\n                low_cpu_mem_usage = True\n\n        if (\n            is_8bit_serializable\n            and quantization_method_from_args == QuantizationMethod.BITS_AND_BYTES\n            and load_in_8bit\n        ):\n            if quantization_method_from_config == QuantizationMethod.BITS_AND_BYTES:\n                logger.warning(\n                    "You passed `quantization_config` to `from_pretrained` but the model you\'re loading already has a"\n                    " `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the"\n                    " one you passed to `from_pretrained`."\n                )\n            config.quantization_config = quantization_config\n        elif (\n            is_8bit_serializable\n            and not load_in_8bit\n            and quantization_method_from_config == QuantizationMethod.BITS_AND_BYTES\n        ):\n            quantization_config = config.quantization_config\n            if isinstance(quantization_config, dict):\n                quantization_config = BitsAndBytesConfig.from_dict(quantization_config, return_unused_kwargs=False)\n            elif isinstance(quantization_config, BitsAndBytesConfig):\n                pass\n            else:\n                raise ValueError(\n                    f"Invalid type for `quantization_config`: {type(quantization_config)}. Should be a `dict` or a"\n                    " `BitsAndBytesConfig` instance."\n                )\n\n            load_in_8bit = quantization_config.load_in_8bit\n\n            if load_in_8bit:\n                if torch_dtype is None:\n                    torch_dtype = torch.float16\n                if device_map is None:\n                    if torch.cuda.is_available():\n                        device_map = {"": torch.cuda.current_device()}\n                    else:\n                        raise RuntimeError("No GPU found. A GPU is needed for quantization.")\n                    logger.info(\n                        "The device_map was not initialized. "\n                        "Setting device_map to {\'\':torch.cuda.current_device()}. "\n                        "If you want to use the model for inference, please set device_map =\'auto\' "\n                    )\n                    if low_cpu_mem_usage is None:\n                        low_cpu_mem_usage = True\n\n        elif (\n            not is_8bit_serializable\n            and not load_in_8bit\n            and quantization_method_from_config == QuantizationMethod.BITS_AND_BYTES\n        ):\n            logger.warning(\n                "Detected the presence of a `quantization_config` attribute in the model\'s configuration but you don\'t have the correct"\n                " `bitsandbytes` version to support int8 serialization. Please install the latest version of `bitsandbytes` with "\n                " `pip install --upgrade bitsandbytes`."\n            )\n\n        # This variable will flag if we\'re loading a sharded checkpoint. In this case the archive file is just the\n        # index of the files.\n        is_sharded = False\n        sharded_metadata = None\n        # Load model\n        loading_info = None\n\n        # Keep in fp32 modules\n        keep_in_fp32_modules = None\n        use_keep_in_fp32_modules = False\n\n        if pretrained_model_name_or_path is not None:\n            pretrained_model_name_or_path = str(pretrained_model_name_or_path)\n            is_local = os.path.isdir(pretrained_model_name_or_path)\n            if is_local:\n                if from_tf and os.path.isfile(\n                    os.path.join(pretrained_model_name_or_path, subfolder, TF_WEIGHTS_NAME + ".index")\n                ):\n                    # Load from a TF 1.0 checkpoint in priority if from_tf\n                    archive_file = os.path.join(pretrained_model_name_or_path, subfolder, TF_WEIGHTS_NAME + ".index")\n                elif from_tf and os.path.isfile(\n                    os.path.join(pretrained_model_name_or_path, subfolder, TF2_WEIGHTS_NAME)\n                ):\n                    # Load from a TF 2.0 checkpoint in priority if from_tf\n                    archive_file = os.path.join(pretrained_model_name_or_path, subfolder, TF2_WEIGHTS_NAME)\n                elif from_flax and os.path.isfile(\n                    os.path.join(pretrained_model_name_or_path, subfolder, FLAX_WEIGHTS_NAME)\n                ):\n                    # Load from a Flax checkpoint in priority if from_flax\n                    archive_file = os.path.join(pretrained_model_name_or_path, subfolder, FLAX_WEIGHTS_NAME)\n                elif use_safetensors is not False and os.path.isfile(\n                    os.path.join(pretrained_model_name_or_path, subfolder, _add_variant(SAFE_WEIGHTS_NAME, variant))\n                ):\n                    # Load from a safetensors checkpoint\n                    archive_file = os.path.join(\n                        pretrained_model_name_or_path, subfolder, _add_variant(SAFE_WEIGHTS_NAME, variant)\n                    )\n                elif use_safetensors is not False and os.path.isfile(\n                    os.path.join(\n                        pretrained_model_name_or_path, subfolder, _add_variant(SAFE_WEIGHTS_INDEX_NAME, variant)\n                    )\n                ):\n                    # Load from a sharded safetensors checkpoint\n                    archive_file = os.path.join(\n                        pretrained_model_name_or_path, subfolder, _add_variant(SAFE_WEIGHTS_INDEX_NAME, variant)\n                    )\n                    is_sharded = True\n                elif os.path.isfile(\n                    os.path.join(pretrained_model_name_or_path, subfolder, _add_variant(WEIGHTS_NAME, variant))\n                ):\n                    # Load from a PyTorch checkpoint\n                    archive_file = os.path.join(\n                        pretrained_model_name_or_path, subfolder, _add_variant(WEIGHTS_NAME, variant)\n                    )\n                elif os.path.isfile(\n                    os.path.join(pretrained_model_name_or_path, subfolder, _add_variant(WEIGHTS_INDEX_NAME, variant))\n                ):\n                    # Load from a sharded PyTorch checkpoint\n                    archive_file = os.path.join(\n                        pretrained_model_name_or_path, subfolder, _add_variant(WEIGHTS_INDEX_NAME, variant)\n                    )\n                    is_sharded = True\n                # At this stage we don\'t have a weight file so we will raise an error.\n                elif os.path.isfile(\n                    os.path.join(pretrained_model_name_or_path, subfolder, TF_WEIGHTS_NAME + ".index")\n                ) or os.path.isfile(os.path.join(pretrained_model_name_or_path, subfolder, TF2_WEIGHTS_NAME)):\n                    raise EnvironmentError(\n                        f"Error no file named {_add_variant(WEIGHTS_NAME, variant)} found in directory"\n                        f" {pretrained_model_name_or_path} but there is a file for TensorFlow weights. Use"\n                        " `from_tf=True` to load this model from those weights."\n                    )\n                elif os.path.isfile(os.path.join(pretrained_model_name_or_path, subfolder, FLAX_WEIGHTS_NAME)):\n                    raise EnvironmentError(\n                        f"Error no file named {_add_variant(WEIGHTS_NAME, variant)} found in directory"\n                        f" {pretrained_model_name_or_path} but there is a file for Flax weights. Use `from_flax=True`"\n                        " to load this model from those weights."\n                    )\n                elif use_safetensors:\n                    raise EnvironmentError(\n                        f"Error no file named {_add_variant(SAFE_WEIGHTS_NAME, variant)} found in directory"\n                        f" {pretrained_model_name_or_path}."\n                    )\n                else:\n                    raise EnvironmentError(\n                        f"Error no file named {_add_variant(WEIGHTS_NAME, variant)}, {TF2_WEIGHTS_NAME},"\n                        f" {TF_WEIGHTS_NAME + \'.index\'} or {FLAX_WEIGHTS_NAME} found in directory"\n                        f" {pretrained_model_name_or_path}."\n                    )\n            elif os.path.isfile(os.path.join(subfolder, pretrained_model_name_or_path)):\n                archive_file = pretrained_model_name_or_path\n                is_local = True\n            elif os.path.isfile(os.path.join(subfolder, pretrained_model_name_or_path + ".index")):\n                if not from_tf:\n                    raise ValueError(\n                        f"We found a TensorFlow checkpoint at {pretrained_model_name_or_path + \'.index\'}, please set "\n                        "from_tf to True to load from this checkpoint."\n                    )\n                archive_file = os.path.join(subfolder, pretrained_model_name_or_path + ".index")\n                is_local = True\n            elif is_remote_url(pretrained_model_name_or_path):\n                filename = pretrained_model_name_or_path\n                resolved_archive_file = download_url(pretrained_model_name_or_path)\n            else:\n                # set correct filename\n                if from_tf:\n                    filename = TF2_WEIGHTS_NAME\n                elif from_flax:\n                    filename = FLAX_WEIGHTS_NAME\n                elif use_safetensors is not False:\n                    filename = _add_variant(SAFE_WEIGHTS_NAME, variant)\n                else:\n                    filename = _add_variant(WEIGHTS_NAME, variant)\n\n                try:\n                    # Load from URL or cache if already cached\n                    cached_file_kwargs = {\n                        "cache_dir": cache_dir,\n                        "force_download": force_download,\n                        "proxies": proxies,\n                        "resume_download": resume_download,\n                        "local_files_only": local_files_only,\n                        "token": token,\n                        "user_agent": user_agent,\n                        "revision": revision,\n                        "subfolder": subfolder,\n                        "_raise_exceptions_for_missing_entries": False,\n                        "_commit_hash": commit_hash,\n                    }\n                    resolved_archive_file = cached_file(pretrained_model_name_or_path, filename, **cached_file_kwargs)\n\n                    # Since we set _raise_exceptions_for_missing_entries=False, we don\'t get an exception but a None\n                    # result when internet is up, the repo and revision exist, but the file does not.\n                    if resolved_archive_file is None and filename == _add_variant(SAFE_WEIGHTS_NAME, variant):\n                        # Maybe the checkpoint is sharded, we try to grab the index name in this case.\n                        resolved_archive_file = cached_file(\n                            pretrained_model_name_or_path,\n                            _add_variant(SAFE_WEIGHTS_INDEX_NAME, variant),\n                            **cached_file_kwargs,\n                        )\n                        if resolved_archive_file is not None:\n                            is_sharded = True\n                        elif use_safetensors:\n                            raise EnvironmentError(\n                                f" {_add_variant(SAFE_WEIGHTS_NAME, variant)} or {_add_variant(SAFE_WEIGHTS_INDEX_NAME, variant)} and thus cannot be loaded with `safetensors`. Please make sure that the model has been saved with `safe_serialization=True` or do not set `use_safetensors=True`."\n                            )\n                        else:\n                            # This repo has no safetensors file of any kind, we switch to PyTorch.\n                            filename = _add_variant(WEIGHTS_NAME, variant)\n                            resolved_archive_file = cached_file(\n                                pretrained_model_name_or_path, filename, **cached_file_kwargs\n                            )\n                    if resolved_archive_file is None and filename == _add_variant(WEIGHTS_NAME, variant):\n                        # Maybe the checkpoint is sharded, we try to grab the index name in this case.\n                        resolved_archive_file = cached_file(\n                            pretrained_model_name_or_path,\n                            _add_variant(WEIGHTS_INDEX_NAME, variant),\n                            **cached_file_kwargs,\n                        )\n                        if resolved_archive_file is not None:\n                            is_sharded = True\n                    if resolved_archive_file is None:\n                        # Otherwise, maybe there is a TF or Flax model file.  We try those to give a helpful error\n                        # message.\n                        has_file_kwargs = {\n                            "revision": revision,\n                            "proxies": proxies,\n                            "token": token,\n                        }\n                        if has_file(pretrained_model_name_or_path, TF2_WEIGHTS_NAME, **has_file_kwargs):\n                            raise EnvironmentError(\n                                f"{pretrained_model_name_or_path} does not appear to have a file named"\n                                f" {_add_variant(WEIGHTS_NAME, variant)} but there is a file for TensorFlow weights."\n                                " Use `from_tf=True` to load this model from those weights."\n                            )\n                        elif has_file(pretrained_model_name_or_path, FLAX_WEIGHTS_NAME, **has_file_kwargs):\n                            raise EnvironmentError(\n                                f"{pretrained_model_name_or_path} does not appear to have a file named"\n                                f" {_add_variant(WEIGHTS_NAME, variant)} but there is a file for Flax weights. Use"\n                                " `from_flax=True` to load this model from those weights."\n                            )\n                        elif variant is not None and has_file(\n                            pretrained_model_name_or_path, WEIGHTS_NAME, **has_file_kwargs\n                        ):\n                            raise EnvironmentError(\n                                f"{pretrained_model_name_or_path} does not appear to have a file named"\n                                f" {_add_variant(WEIGHTS_NAME, variant)} but there is a file without the variant"\n                                f" {variant}. Use `variant=None` to load this model from those weights."\n                            )\n                        else:\n                            raise EnvironmentError(\n                                f"{pretrained_model_name_or_path} does not appear to have a file named"\n                                f" {_add_variant(WEIGHTS_NAME, variant)}, {TF2_WEIGHTS_NAME}, {TF_WEIGHTS_NAME} or"\n                                f" {FLAX_WEIGHTS_NAME}."\n                            )\n                except EnvironmentError:\n                    # Raise any environment error raise by `cached_file`. It will have a helpful error message adapted\n                    # to the original exception.\n                    raise\n                except Exception:\n                    # For any other exception, we throw a generic error.\n                    raise EnvironmentError(\n                        f"Can\'t load the model for \'{pretrained_model_name_or_path}\'. If you were trying to load it"\n                        " from \'https://huggingface.co/models\', make sure you don\'t have a local directory with the"\n                        f" same name. Otherwise, make sure \'{pretrained_model_name_or_path}\' is the correct path to a"\n                        f" directory containing a file named {_add_variant(WEIGHTS_NAME, variant)},"\n                        f" {TF2_WEIGHTS_NAME}, {TF_WEIGHTS_NAME} or {FLAX_WEIGHTS_NAME}."\n                    )\n\n            if is_local:\n                logger.info(f"loading weights file {archive_file}")\n                resolved_archive_file = archive_file\n            else:\n                logger.info(f"loading weights file {filename} from cache at {resolved_archive_file}")\n        else:\n            resolved_archive_file = None\n\n        # We\'ll need to download and cache each checkpoint shard if the checkpoint is sharded.\n        if is_sharded:\n            # rsolved_archive_file becomes a list of files that point to the different checkpoint shards in this case.\n            resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(\n                pretrained_model_name_or_path,\n                resolved_archive_file,\n                cache_dir=cache_dir,\n                force_download=force_download,\n                proxies=proxies,\n                resume_download=resume_download,\n                local_files_only=local_files_only,\n                token=token,\n                user_agent=user_agent,\n                revision=revision,\n                subfolder=subfolder,\n                _commit_hash=commit_hash,\n            )\n\n        if (\n            is_safetensors_available()\n            and isinstance(resolved_archive_file, str)\n            and resolved_archive_file.endswith(".safetensors")\n        ):\n            with safe_open(resolved_archive_file, framework="pt") as f:\n                metadata = f.metadata()\n\n            if metadata.get("format") == "pt":\n                pass\n            elif metadata.get("format") == "tf":\n                from_tf = True\n                logger.info("A TensorFlow safetensors file is being loaded in a PyTorch model.")\n            elif metadata.get("format") == "flax":\n                from_flax = True\n                logger.info("A Flax safetensors file is being loaded in a PyTorch model.")\n            else:\n                raise ValueError(\n                    f"Incompatible safetensors file. File metadata is not [\'pt\', \'tf\', \'flax\'] but {metadata.get(\'format\')}"\n                )\n\n        from_pt = not (from_tf | from_flax)\n\n        # load pt weights early so that we know which dtype to init the model under\n        if from_pt:\n            if not is_sharded and state_dict is None:\n                # Time to load the checkpoint\n                state_dict = load_state_dict(resolved_archive_file)\n\n            # set dtype to instantiate the model under:\n            # 1. If torch_dtype is not None, we use that dtype\n            # 2. If torch_dtype is "auto", we auto-detect dtype from the loaded state_dict, by checking its first\n            #    weights entry that is of a floating type - we assume all floating dtype weights are of the same dtype\n            # we also may have config.torch_dtype available, but we won\'t rely on it till v5\n            dtype_orig = None\n\n            if torch_dtype is not None:\n                if isinstance(torch_dtype, str):\n                    if torch_dtype == "auto":\n                        if hasattr(config, "torch_dtype") and config.torch_dtype is not None:\n                            torch_dtype = config.torch_dtype\n                            logger.info(f"Will use torch_dtype={torch_dtype} as defined in model\'s config object")\n                        else:\n                            if is_sharded and "dtype" in sharded_metadata:\n                                torch_dtype = sharded_metadata["dtype"]\n                            elif not is_sharded:\n                                torch_dtype = get_state_dict_dtype(state_dict)\n                            else:\n                                one_state_dict = load_state_dict(resolved_archive_file[0])\n                                torch_dtype = get_state_dict_dtype(one_state_dict)\n                                del one_state_dict  # free CPU memory\n                            logger.info(\n                                "Since the `torch_dtype` attribute can\'t be found in model\'s config object, "\n                                "will use torch_dtype={torch_dtype} as derived from model\'s weights"\n                            )\n                    else:\n                        raise ValueError(\n                            f\'`torch_dtype` can be either `torch.dtype` or `"auto"`, but received {torch_dtype}\'\n                        )\n                dtype_orig = cls._set_default_torch_dtype(torch_dtype)\n\n            # Check if `_keep_in_fp32_modules` is not None\n            use_keep_in_fp32_modules = (cls._keep_in_fp32_modules is not None) and (\n                torch_dtype == torch.float16 or load_in_4bit or load_in_8bit\n            )\n\n            if is_sharded:\n                loaded_state_dict_keys = sharded_metadata["all_checkpoint_keys"]\n            else:\n                loaded_state_dict_keys = list(state_dict.keys())\n            if low_cpu_mem_usage or (use_keep_in_fp32_modules and is_accelerate_available()):\n                # In case some weights need to be kept in float32 and accelerate is not installed,\n                # we later on want to take the path where state_dict is not None, that is the one\n                # that do not require accelerate.\n                state_dict = None\n\n        config.name_or_path = pretrained_model_name_or_path\n\n        # Instantiate model.\n        init_contexts = [no_init_weights(_enable=_fast_init)]\n\n        if is_deepspeed_zero3_enabled():\n            import deepspeed\n\n            logger.info("Detected DeepSpeed ZeRO-3: activating zero.init() for this model")\n            init_contexts = [deepspeed.zero.Init(config_dict_or_path=deepspeed_config())] + init_contexts\n        elif load_in_8bit or load_in_4bit or low_cpu_mem_usage:\n            init_contexts.append(init_empty_weights())\n\n        if use_flash_attention_2:\n            config = cls._check_and_enable_flash_attn_2(config, torch_dtype=torch_dtype, device_map=device_map)\n\n        with ContextManagers(init_contexts):\n            model = cls(config, *model_args, **model_kwargs)\n\n        # make sure we use the model\'s config since the __init__ call might have copied it\n        config = model.config\n\n        # Check first if we are `from_pt`\n        if use_keep_in_fp32_modules:\n            if is_accelerate_available():\n                low_cpu_mem_usage = True\n            keep_in_fp32_modules = model._keep_in_fp32_modules\n        else:\n            keep_in_fp32_modules = []\n\n        if load_in_8bit or load_in_4bit:\n            from .integrations import get_keys_to_not_convert, replace_with_bnb_linear\n\n            llm_int8_skip_modules = quantization_config.llm_int8_skip_modules\n            load_in_8bit_fp32_cpu_offload = quantization_config.llm_int8_enable_fp32_cpu_offload\n            if load_in_8bit:\n                logger.info("Detected 8-bit loading: activating 8-bit loading for this model")\n            else:\n                logger.info("Detected 4-bit loading: activating 4-bit loading for this model")\n\n            # We keep some modules such as the lm_head in their original dtype for numerical stability reasons\n            if llm_int8_skip_modules is None:\n                modules_to_not_convert = get_keys_to_not_convert(model)\n            else:\n                modules_to_not_convert = llm_int8_skip_modules\n\n            if not isinstance(modules_to_not_convert, list):\n                modules_to_not_convert = [modules_to_not_convert]\n\n            modules_to_not_convert.extend(keep_in_fp32_modules)\n\n            # Extend the modules to not convert to keys that are supposed to be offloaded to `cpu` or `disk`\n            if isinstance(device_map, dict) and len(device_map.keys()) > 1:\n                keys_on_cpu = [key for key, value in device_map.items() if value in ["disk", "cpu"]]\n\n                if len(keys_on_cpu) > 0 and not load_in_8bit_fp32_cpu_offload:\n                    raise ValueError(\n                        "If you want to offload some keys to `cpu` or `disk`, you need to set "\n                        "`llm_int8_enable_fp32_cpu_offload=True`. Note that these modules will not be "\n                        " converted to 8-bit but kept in 32-bit."\n                    )\n\n                modules_to_not_convert.extend(keys_on_cpu)\n\n            supports_4bit = version.parse(importlib.metadata.version("bitsandbytes")) >= version.parse("0.39.0")\n\n            if load_in_4bit and not supports_4bit:\n                raise ValueError(\n                    "You have a version of `bitsandbytes` that is not compatible with 4bit inference and training"\n                    " make sure you have the latest version of `bitsandbytes` installed"\n                )\n\n            model = replace_with_bnb_linear(\n                model, modules_to_not_convert=modules_to_not_convert, quantization_config=quantization_config\n            )\n            # training in 8-bit is only available in 0.37.0+\n            model._is_quantized_training_enabled = version.parse(\n                importlib.metadata.version("bitsandbytes")\n            ) >= version.parse("0.37.0")\n\n            config.quantization_config = quantization_config\n            model.is_8bit_serializable = is_8bit_serializable\n\n        if load_in_8bit and torch_dtype is None:\n            logger.warning(\n                "You are loading your model in 8bit but you did not specify a `torch_dtype` attribute. "\n                "All non-linear modules will be loaded in full precision."\n                " If you want to load the other modules in other precision, please specify a `torch_dtype` attribute."\n            )\n        if quantization_method_from_config == QuantizationMethod.GPTQ:\n            model = quantizer.convert_model(model)\n            model._is_quantized_training_enabled = True\n        elif quantization_method_from_config == QuantizationMethod.AWQ:\n            from .integrations import get_keys_to_not_convert, replace_with_awq_linear\n\n            modules_to_not_convert = get_keys_to_not_convert(model)\n\n            if quantization_config is None:\n                quantization_config = AwqConfig.from_dict(config.quantization_config)\n\n            model, has_been_replaced = replace_with_awq_linear(\n                model, quantization_config=quantization_config, modules_to_not_convert=modules_to_not_convert\n            )\n            model._is_quantized_training_enabled = False\n\n            if not has_been_replaced:\n                logger.warning(\n                    "You are loading an AWQ model but no linear modules were found in your model."\n                    " Please double check your model architecture, or submit an issue on github if you think this is"\n                    " a bug."\n                )\n\n        if quantization_method_from_config is not None:\n            model.quantization_method = quantization_method_from_config\n        elif quantization_method_from_args is not None:\n            model.quantization_method = quantization_method_from_args\n        if hasattr(model, "quantization_method"):\n            model.is_quantized = True\n\n            # We store the original dtype for quantized models as we cannot easily retrieve it\n            # once the weights have been quantized\n            # Note that once you have loaded a quantized model, you can\'t change its dtype so this will\n            # remain a single source of truth\n            config._pre_quantization_dtype = torch_dtype\n\n        if isinstance(device_map, str):\n            special_dtypes = {}\n            if load_in_8bit or load_in_4bit:\n                special_dtypes.update(\n                    {\n                        name: torch_dtype\n                        for name, _ in model.named_parameters()\n                        if any(m in name for m in modules_to_not_convert)\n                    }\n                )\n\n            special_dtypes.update(\n                {\n                    name: torch.float32\n                    for name, _ in model.named_parameters()\n                    if any(m in name for m in keep_in_fp32_modules)\n                }\n            )\n\n            target_dtype = torch_dtype\n\n            if load_in_4bit:\n                if version.parse(importlib.metadata.version("accelerate")) > version.parse("0.19.0"):\n                    from accelerate.utils import CustomDtype\n\n                    target_dtype = CustomDtype.INT4\n                else:\n                    raise ValueError(\n                        "You are using `device_map=\'auto\'` on a 4bit loaded version of the model. To automatically compute"\n                        " the appropriate device map, you should upgrade your `accelerate` library, "\n                        "`pip install --upgrade accelerate` or install it from source to support fp4 auto device map "\n                        "calculation. You may encounter unexpected behavior, or pass your own device map"\n                    )\n            elif load_in_8bit:\n                target_dtype = torch.int8\n\n            no_split_modules = model._get_no_split_modules(device_map)\n            if device_map not in ["auto", "balanced", "balanced_low_0", "sequential"]:\n                raise ValueError(\n                    "If passing a string for `device_map`, please choose \'auto\', \'balanced\', \'balanced_low_0\' or "\n                    "\'sequential\'."\n                )\n\n            device_map_kwargs = {"no_split_module_classes": no_split_modules}\n            if "special_dtypes" in inspect.signature(infer_auto_device_map).parameters:\n                device_map_kwargs["special_dtypes"] = special_dtypes\n            elif len(special_dtypes) > 0:\n                logger.warning(\n                    "This model has some weights that should be kept in higher precision, you need to upgrade "\n                    "`accelerate` to properly deal with them (`pip install --upgrade accelerate`)."\n                )\n            if device_map != "sequential":\n                max_memory = get_balanced_memory(\n                    model,\n                    dtype=target_dtype,\n                    low_zero=(device_map == "balanced_low_0"),\n                    max_memory=max_memory,\n                    **device_map_kwargs,\n                )\n            else:\n                max_memory = get_max_memory(max_memory)\n            if getattr(model, "quantization_method", None) == QuantizationMethod.BITS_AND_BYTES:\n                # need more space for buffers that are created during quantization\n                max_memory = {key: val * 0.90 for key, val in max_memory.items()}\n            device_map_kwargs["max_memory"] = max_memory\n\n            # Make sure tied weights are tied before creating the device map.\n            model.tie_weights()\n            device_map = infer_auto_device_map(model, dtype=target_dtype, **device_map_kwargs)\n\n            if load_in_8bit or load_in_4bit:\n                # The LM head / tied weights or any last module can stay on disk / CPU\n                device_map_without_lm_head = {\n                    key: device_map[key] for key in device_map.keys() if key not in modules_to_not_convert\n                }\n                if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_without_lm_head.values():\n                    raise ValueError(\n                        """\n                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit\n                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping\n                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom\n                        `device_map` to `from_pretrained`. Check\n                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu\n                        for more details.\n                        """\n                    )\n                del device_map_without_lm_head\n\n        elif device_map is not None:\n            model.tie_weights()\n            tied_params = find_tied_parameters(model)\n            # check if we don\'t have tied param in different devices\n            check_tied_parameters_on_same_device(tied_params, device_map)\n\n        if from_tf:\n            if resolved_archive_file.endswith(".index"):\n                # Load from a TensorFlow 1.X checkpoint - provided by original authors\n                model = cls.load_tf_weights(model, config, resolved_archive_file[:-6])  # Remove the \'.index\'\n            else:\n                # Load from our TensorFlow 2.0 checkpoints\n                try:\n                    from .modeling_tf_pytorch_utils import load_tf2_checkpoint_in_pytorch_model\n\n                    model, loading_info = load_tf2_checkpoint_in_pytorch_model(\n                        model, resolved_archive_file, allow_missing_keys=True, output_loading_info=True\n                    )\n                except ImportError:\n                    logger.error(\n                        "Loading a TensorFlow model in PyTorch, requires both PyTorch and TensorFlow to be installed."\n                        " Please see https://pytorch.org/ and https://www.tensorflow.org/install/ for installation"\n                        " instructions."\n                    )\n                    raise\n        elif from_flax:\n            try:\n                from .modeling_flax_pytorch_utils import load_flax_checkpoint_in_pytorch_model\n\n                model = load_flax_checkpoint_in_pytorch_model(model, resolved_archive_file)\n            except ImportError:\n                logger.error(\n                    "Loading a Flax model in PyTorch, requires both PyTorch and Flax to be installed. Please see"\n                    " https://pytorch.org/ and https://flax.readthedocs.io/en/latest/installation.html for"\n                    " installation instructions."\n                )\n                raise\n        elif from_pt:\n            # restore default dtype\n            if dtype_orig is not None:\n                torch.set_default_dtype(dtype_orig)\n            (\n                model,\n                missing_keys,\n                unexpected_keys,\n                mismatched_keys,\n                offload_index,\n                error_msgs,\n            ) = cls._load_pretrained_model(\n                model,\n                state_dict,\n                loaded_state_dict_keys,  # XXX: rename?\n                resolved_archive_file,\n                pretrained_model_name_or_path,\n                ignore_mismatched_sizes=ignore_mismatched_sizes,\n                sharded_metadata=sharded_metadata,\n                _fast_init=_fast_init,\n                low_cpu_mem_usage=low_cpu_mem_usage,\n                device_map=device_map,\n                offload_folder=offload_folder,\n                offload_state_dict=offload_state_dict,\n                dtype=torch_dtype,\n                is_quantized=(getattr(model, "quantization_method", None) == QuantizationMethod.BITS_AND_BYTES),\n                keep_in_fp32_modules=keep_in_fp32_modules,\n            )\n\n        model.is_loaded_in_4bit = load_in_4bit\n        model.is_loaded_in_8bit = load_in_8bit\n\n        # make sure token embedding weights are still tied if needed\n        model.tie_weights()\n\n        # Set model in evaluation mode to deactivate DropOut modules by default\n        model.eval()\n\n        # If it is a model with generation capabilities, attempt to load the generation config\n        if model.can_generate() and pretrained_model_name_or_path is not None:\n            try:\n                model.generation_config = GenerationConfig.from_pretrained(\n                    pretrained_model_name_or_path,\n                    cache_dir=cache_dir,\n                    force_download=force_download,\n                    resume_download=resume_download,\n                    proxies=proxies,\n                    local_files_only=local_files_only,\n                    token=token,\n                    revision=revision,\n                    subfolder=subfolder,\n                    _from_auto=from_auto_class,\n                    _from_pipeline=from_pipeline,\n                    **kwargs,\n                )\n            except OSError:\n                logger.info(\n                    "Generation config file not found, using a generation config created from the model config."\n                )\n                pass\n\n        # Dispatch model with hooks on all devices if necessary\n        if device_map is not None:\n            device_map_kwargs = {\n                "device_map": device_map,\n                "offload_dir": offload_folder,\n                "offload_index": offload_index,\n            }\n            if "skip_keys" in inspect.signature(dispatch_model).parameters:\n                device_map_kwargs["skip_keys"] = model._skip_keys_device_placement\n            dispatch_model(model, **device_map_kwargs)\n\n        if quantization_method_from_args == QuantizationMethod.GPTQ:\n            if quantization_config.tokenizer is None:\n                quantization_config.tokenizer = pretrained_model_name_or_path\n            if cls.main_input_name != "input_ids":\n                raise RuntimeError("We can only quantize pure text model.")\n            quantizer.quantize_model(model, quantization_config.tokenizer)\n            config.quantization_config = GPTQConfig.from_dict_optimum(quantizer.to_dict())\n            model._is_quantized_training_enabled = True\n        if quantization_method_from_config == QuantizationMethod.GPTQ:\n            model = quantizer.post_init_model(model)\n\n        if _adapter_model_path is not None:\n            model.load_adapter(\n                _adapter_model_path,\n                adapter_name=adapter_name,\n                token=token,\n                adapter_kwargs=adapter_kwargs,\n            )\n\n        if output_loading_info:\n            if loading_info is None:\n                loading_info = {\n                    "missing_keys": missing_keys,\n                    "unexpected_keys": unexpected_keys,\n                    "mismatched_keys": mismatched_keys,\n                    "error_msgs": error_msgs,\n                }\n            return model, loading_info\n\n        return model\n\n    @classmethod\n    def _load_pretrained_model(\n        cls,\n        model,\n        state_dict,\n        loaded_keys,\n        resolved_archive_file,\n        pretrained_model_name_or_path,\n        ignore_mismatched_sizes=False,\n        sharded_metadata=None,\n        _fast_init=True,\n        low_cpu_mem_usage=False,\n        device_map=None,\n        offload_folder=None,\n        offload_state_dict=None,\n        dtype=None,\n        is_quantized=False,\n        keep_in_fp32_modules=None,\n    ):\n        is_safetensors = False\n        if is_quantized:\n            from .integrations import set_module_quantized_tensor_to_device\n\n        if device_map is not None and "disk" in device_map.values():\n            archive_file = (\n                resolved_archive_file[0] if isinstance(resolved_archive_file, (list, tuple)) else resolved_archive_file\n            )\n            is_safetensors = archive_file.endswith(".safetensors")\n            if offload_folder is None and not is_safetensors:\n                raise ValueError(\n                    "The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder`"\n                    " for them. Alternatively, make sure you have `safetensors` installed if the model you are using"\n                    " offers the weights in this format."\n                )\n            if offload_folder is not None:\n                os.makedirs(offload_folder, exist_ok=True)\n            if offload_state_dict is None:\n                offload_state_dict = True\n\n        is_sharded_safetensors = is_safetensors and sharded_metadata is not None\n\n        # tie the model weights before retrieving the state_dict\n        model.tie_weights()\n\n        # Retrieve missing & unexpected_keys\n        model_state_dict = model.state_dict()\n        expected_keys = list(model_state_dict.keys())\n        prefix = model.base_model_prefix\n\n        def _fix_key(key):\n            if "beta" in key:\n                return key.replace("beta", "bias")\n            if "gamma" in key:\n                return key.replace("gamma", "weight")\n            return key\n\n        original_loaded_keys = loaded_keys\n        loaded_keys = [_fix_key(key) for key in loaded_keys]\n\n        if len(prefix) > 0:\n            has_prefix_module = any(s.startswith(prefix) for s in loaded_keys)\n            expects_prefix_module = any(s.startswith(prefix) for s in expected_keys)\n        else:\n            has_prefix_module = False\n            expects_prefix_module = False\n\n        # key re-naming operations are never done on the keys\n        # that are loaded, but always on the keys of the newly initialized model\n        remove_prefix_from_model = not has_prefix_module and expects_prefix_module\n        add_prefix_to_model = has_prefix_module and not expects_prefix_module\n\n        if remove_prefix_from_model:\n            _prefix = f"{prefix}."\n            expected_keys_not_prefixed = [s for s in expected_keys if not s.startswith(_prefix)]\n            expected_keys = [s[len(_prefix) :] if s.startswith(_prefix) else s for s in expected_keys]\n        elif add_prefix_to_model:\n            expected_keys = [".".join([prefix, s]) for s in expected_keys]\n\n        missing_keys = list(set(expected_keys) - set(loaded_keys))\n        unexpected_keys = set(loaded_keys) - set(expected_keys)\n        # Remove nonpersistent buffers from unexpected keys: they are not in the state dict but will be in the model\n        # buffers\n        model_buffers = {n for n, _ in model.named_buffers()}\n        if remove_prefix_from_model:\n            model_buffers = {key[len(_prefix) :] if key.startswith(_prefix) else key for key in model_buffers}\n        elif add_prefix_to_model:\n            model_buffers = {".".join([prefix, key]) for key in model_buffers}\n        unexpected_keys = list(unexpected_keys - model_buffers)\n\n        model.tie_weights()\n        if device_map is None and not is_fsdp_enabled():\n            ptrs = collections.defaultdict(list)\n            for name, tensor in model.state_dict().items():\n                id_tensor = id_tensor_storage(tensor)\n                ptrs[id_tensor].append(name)\n\n            # These are all the pointers of shared tensors.\n            tied_params = [names for _, names in ptrs.items() if len(names) > 1]\n        else:\n            # id function doesn\'t work for meta tensor so we need this function\n            tied_params = find_tied_parameters(model)\n\n        for group in tied_params:\n            if remove_prefix_from_model:\n                group = [key[len(_prefix) :] if key.startswith(_prefix) else key for key in group]\n            elif add_prefix_to_model:\n                group = [".".join([prefix, key]) for key in group]\n            missing_in_group = [k for k in missing_keys if k in group]\n            if len(missing_in_group) > 0 and len(missing_in_group) < len(group):\n                missing_keys = [k for k in missing_keys if k not in missing_in_group]\n\n        # Some models may have keys that are not in the state by design, removing them before needlessly warning\n        # the user.\n        if cls._keys_to_ignore_on_load_missing is not None:\n            for pat in cls._keys_to_ignore_on_load_missing:\n                missing_keys = [k for k in missing_keys if re.search(pat, k) is None]\n\n        if cls._keys_to_ignore_on_load_unexpected is not None:\n            for pat in cls._keys_to_ignore_on_load_unexpected:\n                unexpected_keys = [k for k in unexpected_keys if re.search(pat, k) is None]\n\n        # retrieve weights on meta device and put them back on CPU.\n        # This is not ideal in terms of memory, but if we don\'t do that not, we can\'t initialize them in the next step\n        if low_cpu_mem_usage:\n            for key in missing_keys:\n                if key in list(model_state_dict.keys()):\n                    key = key\n                elif f"{prefix}.{key}" in list(model_state_dict.keys()):\n                    key = f"{prefix}.{key}"\n                elif key.startswith(prefix) and ".".join(key.split(".")[1:]) in list(model_state_dict.keys()):\n                    key = ".".join(key.split(".")[1:])\n                param = model_state_dict[key]\n\n                # upcast in fp32 if any\n                target_dtype = dtype\n                if (\n                    keep_in_fp32_modules is not None\n                    and dtype == torch.float16\n                    and any(\n                        module_to_keep_in_fp32 in key.split(".") for module_to_keep_in_fp32 in keep_in_fp32_modules\n                    )\n                ):\n                    target_dtype = torch.float32\n\n                if param.device == torch.device("meta"):\n                    if not (is_quantized):\n                        set_module_tensor_to_device(model, key, "cpu", torch.empty(*param.size(), dtype=target_dtype))\n                    else:\n                        set_module_quantized_tensor_to_device(\n                            model, key, "cpu", torch.empty(*param.size(), dtype=target_dtype)\n                        )\n\n        # retrieve unintialized modules and initialize before maybe overriding that with the pretrained weights.\n        if _fast_init:\n            if remove_prefix_from_model:\n                _loaded_keys = [f"{prefix}.{k}" for k in loaded_keys]\n            elif add_prefix_to_model:\n                _loaded_keys = [k[len(prefix) + 1 :] for k in loaded_keys]\n            else:\n                _loaded_keys = loaded_keys\n            set_initialized_submodules(model, _loaded_keys)\n            # This will only initialize submodules that are not marked as initialized by the line above.\n            model.apply(model._initialize_weights)\n\n        # Set some modules to fp32 if any\n        if keep_in_fp32_modules is not None:\n            for name, param in model.named_parameters():\n                if any(module_to_keep_in_fp32 in name.split(".") for module_to_keep_in_fp32 in keep_in_fp32_modules):\n                    # param = param.to(torch.float32) does not work here as only in the local scope.\n                    param.data = param.data.to(torch.float32)\n\n        # Make sure we are able to load base models as well as derived models (with heads)\n        start_prefix = ""\n        model_to_load = model\n        if len(cls.base_model_prefix) > 0 and not hasattr(model, cls.base_model_prefix) and has_prefix_module:\n            start_prefix = cls.base_model_prefix + "."\n        if len(cls.base_model_prefix) > 0 and hasattr(model, cls.base_model_prefix) and not has_prefix_module:\n            model_to_load = getattr(model, cls.base_model_prefix)\n            base_model_expected_keys = list(model_to_load.state_dict().keys())\n            if any(key in expected_keys_not_prefixed and key not in base_model_expected_keys for key in loaded_keys):\n                raise ValueError(\n                    "The state dictionary of the model you are trying to load is corrupted. Are you sure it was "\n                    "properly saved?"\n                )\n            if device_map is not None:\n                device_map = {k.replace(f"{cls.base_model_prefix}.", ""): v for k, v in device_map.items()}\n\n        def _find_mismatched_keys(\n            state_dict,\n            model_state_dict,\n            loaded_keys,\n            add_prefix_to_model,\n            remove_prefix_from_model,\n            ignore_mismatched_sizes,\n        ):\n            mismatched_keys = []\n            if ignore_mismatched_sizes:\n                for checkpoint_key in loaded_keys:\n                    # If the checkpoint is sharded, we may not have the key here.\n                    if checkpoint_key not in state_dict:\n                        continue\n                    model_key = checkpoint_key\n                    if remove_prefix_from_model:\n                        # The model key starts with `prefix` but `checkpoint_key` doesn\'t so we add it.\n                        model_key = f"{prefix}.{checkpoint_key}"\n                    elif add_prefix_to_model:\n                        # The model key doesn\'t start with `prefix` but `checkpoint_key` does so we remove it.\n                        model_key = ".".join(checkpoint_key.split(".")[1:])\n\n                    if (\n                        model_key in model_state_dict\n                        and state_dict[checkpoint_key].shape != model_state_dict[model_key].shape\n                    ):\n                        mismatched_keys.append(\n                            (checkpoint_key, state_dict[checkpoint_key].shape, model_state_dict[model_key].shape)\n                        )\n                        del state_dict[checkpoint_key]\n            return mismatched_keys\n\n        if resolved_archive_file is not None:\n            folder = os.path.sep.join(resolved_archive_file[0].split(os.path.sep)[:-1])\n        else:\n            folder = None\n        if device_map is not None and is_safetensors:\n            param_device_map = expand_device_map(device_map, original_loaded_keys, start_prefix)\n            str_dtype = str(dtype).replace("torch.", "") if dtype is not None else "float32"\n            if sharded_metadata is None:\n                archive_file = (\n                    resolved_archive_file[0]\n                    if isinstance(resolved_archive_file, (list, tuple))\n                    else resolved_archive_file\n                )\n                weight_map = {p: archive_file for p in original_loaded_keys}\n            else:\n                weight_map = {p: os.path.join(folder, f) for p, f in sharded_metadata["weight_map"].items()}\n            offload_index = {\n                p[len(start_prefix) :]: {"safetensors_file": f, "weight_name": p, "dtype": str_dtype}\n                for p, f in weight_map.items()\n                if p.startswith(start_prefix) and param_device_map[p[len(start_prefix) :]] == "disk"\n            }\n\n        if state_dict is not None:\n            # Whole checkpoint\n            mismatched_keys = _find_mismatched_keys(\n                state_dict,\n                model_state_dict,\n                original_loaded_keys,\n                add_prefix_to_model,\n                remove_prefix_from_model,\n                ignore_mismatched_sizes,\n            )\n            error_msgs = _load_state_dict_into_model(model_to_load, state_dict, start_prefix)\n            offload_index = None\n        else:\n            # Sharded checkpoint or whole but low_cpu_mem_usage==True\n\n            # This should always be a list but, just to be sure.\n            if not isinstance(resolved_archive_file, list):\n                resolved_archive_file = [resolved_archive_file]\n\n            error_msgs = []\n            mismatched_keys = []\n            if not is_safetensors:\n                offload_index = {} if device_map is not None and "disk" in device_map.values() else None\n            if offload_state_dict:\n                state_dict_folder = tempfile.mkdtemp()\n                state_dict_index = {}\n            else:\n                state_dict_folder = None\n                state_dict_index = None\n\n            if is_sharded_safetensors:\n                disk_only_shard_files = get_disk_only_shard_files(\n                    device_map, sharded_metadata=sharded_metadata, start_prefix=start_prefix\n                )\n                disk_only_shard_files = [os.path.join(folder, f) for f in disk_only_shard_files]\n            else:\n                disk_only_shard_files = []\n\n            if len(resolved_archive_file) > 1:\n                resolved_archive_file = logging.tqdm(resolved_archive_file, desc="Loading checkpoint shards")\n            for shard_file in resolved_archive_file:\n                # Skip the load for shards that only contain disk-offloaded weights when using safetensors for the offload.\n                if shard_file in disk_only_shard_files:\n                    continue\n                state_dict = load_state_dict(shard_file)\n\n                # Mistmatched keys contains tuples key/shape1/shape2 of weights in the checkpoint that have a shape not\n                # matching the weights in the model.\n                mismatched_keys += _find_mismatched_keys(\n                    state_dict,\n                    model_state_dict,\n                    original_loaded_keys,\n                    add_prefix_to_model,\n                    remove_prefix_from_model,\n                    ignore_mismatched_sizes,\n                )\n                if low_cpu_mem_usage:\n                    if not is_fsdp_enabled() or is_fsdp_enabled_and_dist_rank_0():\n                        new_error_msgs, offload_index, state_dict_index = _load_state_dict_into_meta_model(\n                            model_to_load,\n                            state_dict,\n                            loaded_keys,\n                            start_prefix,\n                            expected_keys,\n                            device_map=device_map,\n                            offload_folder=offload_folder,\n                            offload_index=offload_index,\n                            state_dict_folder=state_dict_folder,\n                            state_dict_index=state_dict_index,\n                            dtype=dtype,\n                            is_quantized=is_quantized,\n                            is_safetensors=is_safetensors,\n                            keep_in_fp32_modules=keep_in_fp32_modules,\n                        )\n                        error_msgs += new_error_msgs\n                    else:\n                        for key, param in model_to_load.state_dict().items():\n                            if param.device == torch.device("meta"):\n                                if not (is_quantized):\n                                    set_module_tensor_to_device(\n                                        model_to_load, key, "cpu", torch.empty(*param.size(), dtype=dtype)\n                                    )\n                                else:\n                                    set_module_quantized_tensor_to_device(\n                                        model_to_load, key, "cpu", torch.empty(*param.size(), dtype=dtype)\n                                    )\n                else:\n                    error_msgs += _load_state_dict_into_model(model_to_load, state_dict, start_prefix)\n\n                # force memory release\n                del state_dict\n                gc.collect()\n\n            if offload_index is not None and len(offload_index) > 0:\n                if model != model_to_load:\n                    # We need to add the prefix of the base model\n                    prefix = cls.base_model_prefix\n                    if not is_safetensors:\n                        for weight_name in offload_index:\n                            shutil.move(\n                                os.path.join(offload_folder, f"{weight_name}.dat"),\n                                os.path.join(offload_folder, f"{prefix}.{weight_name}.dat"),\n                            )\n                    offload_index = {f"{prefix}.{key}": value for key, value in offload_index.items()}\n                if not is_safetensors:\n                    save_offload_index(offload_index, offload_folder)\n                    offload_index = None\n\n            if offload_state_dict:\n                # Load back temporarily offloaded state dict\n                load_offloaded_weights(model_to_load, state_dict_index, state_dict_folder)\n                shutil.rmtree(state_dict_folder)\n\n        if len(error_msgs) > 0:\n            error_msg = "\\n\\t".join(error_msgs)\n            if "size mismatch" in error_msg:\n                error_msg += (\n                    "\\n\\tYou may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method."\n                )\n            raise RuntimeError(f"Error(s) in loading state_dict for {model.__class__.__name__}:\\n\\t{error_msg}")\n\n        if is_quantized:\n            unexpected_keys = [elem for elem in unexpected_keys if "SCB" not in elem]\n            missing_keys = [elem for elem in missing_keys if "SCB" not in elem]\n\n        if len(unexpected_keys) > 0:\n            archs = [] if model.config.architectures is None else model.config.architectures\n            warner = logger.warning if model.__class__.__name__ in archs else logger.info\n            warner(\n                f"Some weights of the model checkpoint at {pretrained_model_name_or_path} were not used when"\n                f" initializing {model.__class__.__name__}: {unexpected_keys}\\n- This IS expected if you are"\n                f" initializing {model.__class__.__name__} from the checkpoint of a model trained on another task or"\n                " with another architecture (e.g. initializing a BertForSequenceClassification model from a"\n                " BertForPreTraining model).\\n- This IS NOT expected if you are initializing"\n                f" {model.__class__.__name__} from the checkpoint of a model that you expect to be exactly identical"\n                " (initializing a BertForSequenceClassification model from a BertForSequenceClassification model)."\n            )\n        else:\n            logger.info(f"All model checkpoint weights were used when initializing {model.__class__.__name__}.\\n")\n        if len(missing_keys) > 0:\n            logger.warning(\n                f"Some weights of {model.__class__.__name__} were not initialized from the model checkpoint at"\n                f" {pretrained_model_name_or_path} and are newly initialized: {missing_keys}\\nYou should probably"\n                " TRAIN this model on a down-stream task to be able to use it for predictions and inference."\n            )\n        elif len(mismatched_keys) == 0:\n            logger.info(\n                f"All the weights of {model.__class__.__name__} were initialized from the model checkpoint at"\n                f" {pretrained_model_name_or_path}.\\nIf your task is similar to the task the model of the checkpoint"\n                f" was trained on, you can already use {model.__class__.__name__} for predictions without further"\n                " training."\n            )\n        if len(mismatched_keys) > 0:\n            mismatched_warning = "\\n".join(\n                [\n                    f"- {key}: found shape {shape1} in the checkpoint and {shape2} in the model instantiated"\n                    for key, shape1, shape2 in mismatched_keys\n                ]\n            )\n            logger.warning(\n                f"Some weights of {model.__class__.__name__} were not initialized from the model checkpoint at"\n                f" {pretrained_model_name_or_path} and are newly initialized because the shapes did not"\n                f" match:\\n{mismatched_warning}\\nYou should probably TRAIN this model on a down-stream task to be able"\n                " to use it for predictions and inference."\n            )\n\n        return model, missing_keys, unexpected_keys, mismatched_keys, offload_index, error_msgs\n\n    def retrieve_modules_from_names(self, names, add_prefix=False, remove_prefix=False):\n        module_keys = {".".join(key.split(".")[:-1]) for key in names}\n\n        # torch.nn.ParameterList is a special case where two parameter keywords\n        # are appended to the module name, *e.g.* bert.special_embeddings.0\n        module_keys = module_keys.union(\n            {".".join(key.split(".")[:-2]) for key in names if len(key) > 0 and key[-1].isdigit()}\n        )\n\n        retrieved_modules = []\n        # retrieve all modules that has at least one missing weight name\n        for name, module in self.named_modules():\n            if remove_prefix:\n                _prefix = f"{self.base_model_prefix}."\n                name = name[len(_prefix) :] if name.startswith(_prefix) else name\n            elif add_prefix:\n                name = ".".join([self.base_model_prefix, name]) if len(name) > 0 else self.base_model_prefix\n\n            if name in module_keys:\n                retrieved_modules.append(module)\n\n        return retrieved_modules\n\n    @staticmethod\n    def _load_pretrained_model_low_mem(model, loaded_state_dict_keys, resolved_archive_file, start_prefix=""):\n        """\n        This is an experimental function that loads the model using ~1.x model size CPU memory\n\n        Before you call it do:\n\n        1. save which state_dict keys are available\n        2. drop state_dict before model is created, since the latter takes 1x model size memory\n\n        Here then we continue:\n\n        3. switch to the meta device all params/buffers that are going to be replaced from the loaded state_dict\n        4. load state_dict 2nd time\n        5. replace the params/buffers from the state_dict\n\n        Currently, it doesn\'t handle missing_keys, unexpected_keys, mismatched_keys. It can\'t handle deepspeed.\n        """\n\n        _move_model_to_meta(model, loaded_state_dict_keys, start_prefix)\n        state_dict = load_state_dict(resolved_archive_file)\n        error_msgs = _load_state_dict_into_meta_model(model, state_dict, loaded_state_dict_keys, start_prefix)\n        return error_msgs\n\n    @classmethod\n    def register_for_auto_class(cls, auto_class="AutoModel"):\n        """\n        Register this class with a given auto class. This should only be used for custom models as the ones in the\n        library are already mapped with an auto class.\n\n        <Tip warning={true}>\n\n        This API is experimental and may have some slight breaking changes in the next releases.\n\n        </Tip>\n\n        Args:\n            auto_class (`str` or `type`, *optional*, defaults to `"AutoModel"`):\n                The auto class to register this new model with.\n        """\n        if not isinstance(auto_class, str):\n            auto_class = auto_class.__name__\n\n        import transformers.models.auto as auto_module\n\n        if not hasattr(auto_module, auto_class):\n            raise ValueError(f"{auto_class} is not a valid auto class.")\n\n        cls._auto_class = auto_class\n\n    def to_bettertransformer(self) -> "PreTrainedModel":\n        """\n        Converts the model to use [PyTorch\'s native attention\n        implementation](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html), integrated to\n        Transformers through [Optimum library](https://huggingface.co/docs/optimum/bettertransformer/overview). Only a\n        subset of all Transformers models are supported.\n\n        PyTorch\'s attention fastpath allows to speed up inference through kernel fusions and the use of [nested\n        tensors](https://pytorch.org/docs/stable/nested.html). Detailed benchmarks can be found in [this blog\n        post](https://medium.com/pytorch/bettertransformer-out-of-the-box-performance-for-huggingface-transformers-3fbe27d50ab2).\n\n        Returns:\n            [`PreTrainedModel`]: The model converted to BetterTransformer.\n        """\n        if not is_optimum_available():\n            raise ImportError("The package `optimum` is required to use Better Transformer.")\n\n        from optimum.version import __version__ as optimum_version\n\n        if version.parse(optimum_version) < version.parse("1.7.0"):\n            raise ImportError(\n                f"Please install optimum>=1.7.0 to use Better Transformer. The version {optimum_version} was found."\n            )\n\n        from optimum.bettertransformer import BetterTransformer\n\n        return BetterTransformer.transform(self)\n\n    def reverse_bettertransformer(self):\n        """\n        Reverts the transformation from [`~PreTrainedModel.to_bettertransformer`] so that the original modeling is\n        used, for example in order to save the model.\n\n        Returns:\n            [`PreTrainedModel`]: The model converted back to the original modeling.\n        """\n        if not is_optimum_available():\n            raise ImportError("The package `optimum` is required to use Better Transformer.")\n\n        from optimum.version import __version__ as optimum_version\n\n        if version.parse(optimum_version) < version.parse("1.7.0"):\n            raise ImportError(\n                f"Please install optimum>=1.7.0 to use Better Transformer. The version {optimum_version} was found."\n            )\n\n        from optimum.bettertransformer import BetterTransformer\n\n        return BetterTransformer.reverse(self)\n\n    def warn_if_padding_and_no_attention_mask(self, input_ids, attention_mask):\n        """\n        Shows a one-time warning if the input_ids appear to contain padding and no attention mask was given.\n        """\n\n        # Skip the check during tracing.\n        if is_torch_fx_proxy(input_ids) or torch.jit.is_tracing() or is_torchdynamo_compiling():\n            return\n\n        if (attention_mask is not None) or (self.config.pad_token_id is None):\n            return\n\n        # Check only the first and last input IDs to reduce overhead.\n        if self.config.pad_token_id in input_ids[:, [-1, 0]]:\n            warn_string = (\n                "We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See "\n                "https://huggingface.co/docs/transformers/troubleshooting"\n                "#incorrect-output-when-padding-tokens-arent-masked."\n            )\n\n            # If the pad token is equal to either BOS, EOS, or SEP, we do not know whether the user should use an\n            # attention_mask or not. In this case, we should still show a warning because this is a rare case.\n            if (\n                (self.config.bos_token_id is not None and self.config.bos_token_id == self.config.pad_token_id)\n                or (self.config.eos_token_id is not None and self.config.eos_token_id == self.config.pad_token_id)\n                or (self.config.sep_token_id is not None and self.config.sep_token_id == self.config.pad_token_id)\n            ):\n                warn_string += (\n                    f"\\nYou may ignore this warning if your `pad_token_id` ({self.config.pad_token_id}) is identical "\n                    f"to the `bos_token_id` ({self.config.bos_token_id}), `eos_token_id` ({self.config.eos_token_id}), "\n                    f"or the `sep_token_id` ({self.config.sep_token_id}), and your input is not padded."\n                )\n\n            logger.warning_once(warn_string)\n\n\nPreTrainedModel.push_to_hub = copy_func(PreTrainedModel.push_to_hub)\nif PreTrainedModel.push_to_hub.__doc__ is not None:\n    PreTrainedModel.push_to_hub.__doc__ = PreTrainedModel.push_to_hub.__doc__.format(\n        object="model", object_class="AutoModel", object_files="model file"\n    )\n\n\nclass PoolerStartLogits(nn.Module):\n    """\n    Compute SQuAD start logits from sequence hidden states.\n\n    Args:\n        config ([`PretrainedConfig`]):\n            The config used by the model, will be used to grab the `hidden_size` of the model.\n    """\n\n    def __init__(self, config: PretrainedConfig):\n        super().__init__()\n        self.dense = nn.Linear(config.hidden_size, 1)\n\n    def forward(\n        self, hidden_states: torch.FloatTensor, p_mask: Optional[torch.FloatTensor] = None\n    ) -> torch.FloatTensor:\n        """\n        Args:\n            hidden_states (`torch.FloatTensor` of shape `(batch_size, seq_len, hidden_size)`):\n                The final hidden states of the model.\n            p_mask (`torch.FloatTensor` of shape `(batch_size, seq_len)`, *optional*):\n                Mask for tokens at invalid position, such as query and special symbols (PAD, SEP, CLS). 1.0 means token\n                should be masked.\n\n        Returns:\n            `torch.FloatTensor`: The start logits for SQuAD.\n        """\n        x = self.dense(hidden_states).squeeze(-1)\n\n        if p_mask is not None:\n            if get_parameter_dtype(self) == torch.float16:\n                x = x * (1 - p_mask) - 65500 * p_mask\n            else:\n                x = x * (1 - p_mask) - 1e30 * p_mask\n\n        return x\n\n\nclass PoolerEndLogits(nn.Module):\n    """\n    Compute SQuAD end logits from sequence hidden states.\n\n    Args:\n        config ([`PretrainedConfig`]):\n            The config used by the model, will be used to grab the `hidden_size` of the model and the `layer_norm_eps`\n            to use.\n    """\n\n    def __init__(self, config: PretrainedConfig):\n        super().__init__()\n        self.dense_0 = nn.Linear(config.hidden_size * 2, config.hidden_size)\n        self.activation = nn.Tanh()\n        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)\n        self.dense_1 = nn.Linear(config.hidden_size, 1)\n\n    def forward(\n        self,\n        hidden_states: torch.FloatTensor,\n        start_states: Optional[torch.FloatTensor] = None,\n        start_positions: Optional[torch.LongTensor] = None,\n        p_mask: Optional[torch.FloatTensor] = None,\n    ) -> torch.FloatTensor:\n        """\n        Args:\n            hidden_states (`torch.FloatTensor` of shape `(batch_size, seq_len, hidden_size)`):\n                The final hidden states of the model.\n            start_states (`torch.FloatTensor` of shape `(batch_size, seq_len, hidden_size)`, *optional*):\n                The hidden states of the first tokens for the labeled span.\n            start_positions (`torch.LongTensor` of shape `(batch_size,)`, *optional*):\n                The position of the first token for the labeled span.\n            p_mask (`torch.FloatTensor` of shape `(batch_size, seq_len)`, *optional*):\n                Mask for tokens at invalid position, such as query and special symbols (PAD, SEP, CLS). 1.0 means token\n                should be masked.\n\n        <Tip>\n\n        One of `start_states` or `start_positions` should be not `None`. If both are set, `start_positions` overrides\n        `start_states`.\n\n        </Tip>\n\n        Returns:\n            `torch.FloatTensor`: The end logits for SQuAD.\n        """\n        assert (\n            start_states is not None or start_positions is not None\n        ), "One of start_states, start_positions should be not None"\n        if start_positions is not None:\n            slen, hsz = hidden_states.shape[-2:]\n            start_positions = start_positions[:, None, None].expand(-1, -1, hsz)  # shape (bsz, 1, hsz)\n            start_states = hidden_states.gather(-2, start_positions)  # shape (bsz, 1, hsz)\n            start_states = start_states.expand(-1, slen, -1)  # shape (bsz, slen, hsz)\n\n        x = self.dense_0(torch.cat([hidden_states, start_states], dim=-1))\n        x = self.activation(x)\n        x = self.LayerNorm(x)\n        x = self.dense_1(x).squeeze(-1)\n\n        if p_mask is not None:\n            if get_parameter_dtype(self) == torch.float16:\n                x = x * (1 - p_mask) - 65500 * p_mask\n            else:\n                x = x * (1 - p_mask) - 1e30 * p_mask\n\n        return x\n\n\nclass PoolerAnswerClass(nn.Module):\n    """\n    Compute SQuAD 2.0 answer class from classification and start tokens hidden states.\n\n    Args:\n        config ([`PretrainedConfig`]):\n            The config used by the model, will be used to grab the `hidden_size` of the model.\n    """\n\n    def __init__(self, config):\n        super().__init__()\n        self.dense_0 = nn.Linear(config.hidden_size * 2, config.hidden_size)\n        self.activation = nn.Tanh()\n        self.dense_1 = nn.Linear(config.hidden_size, 1, bias=False)\n\n    def forward(\n        self,\n        hidden_states: torch.FloatTensor,\n        start_states: Optional[torch.FloatTensor] = None,\n        start_positions: Optional[torch.LongTensor] = None,\n        cls_index: Optional[torch.LongTensor] = None,\n    ) -> torch.FloatTensor:\n        """\n        Args:\n            hidden_states (`torch.FloatTensor` of shape `(batch_size, seq_len, hidden_size)`):\n                The final hidden states of the model.\n            start_states (`torch.FloatTensor` of shape `(batch_size, seq_len, hidden_size)`, *optional*):\n                The hidden states of the first tokens for the labeled span.\n            start_positions (`torch.LongTensor` of shape `(batch_size,)`, *optional*):\n                The position of the first token for the labeled span.\n            cls_index (`torch.LongTensor` of shape `(batch_size,)`, *optional*):\n                Position of the CLS token for each sentence in the batch. If `None`, takes the last token.\n\n        <Tip>\n\n        One of `start_states` or `start_positions` should be not `None`. If both are set, `start_positions` overrides\n        `start_states`.\n\n        </Tip>\n\n        Returns:\n            `torch.FloatTensor`: The SQuAD 2.0 answer class.\n        """\n        # No dependency on end_feature so that we can obtain one single `cls_logits` for each sample.\n        hsz = hidden_states.shape[-1]\n        assert (\n            start_states is not None or start_positions is not None\n        ), "One of start_states, start_positions should be not None"\n        if start_positions is not None:\n            start_positions = start_positions[:, None, None].expand(-1, -1, hsz)  # shape (bsz, 1, hsz)\n            start_states = hidden_states.gather(-2, start_positions).squeeze(-2)  # shape (bsz, hsz)\n\n        if cls_index is not None:\n            cls_index = cls_index[:, None, None].expand(-1, -1, hsz)  # shape (bsz, 1, hsz)\n            cls_token_state = hidden_states.gather(-2, cls_index).squeeze(-2)  # shape (bsz, hsz)\n        else:\n            cls_token_state = hidden_states[:, -1, :]  # shape (bsz, hsz)\n\n        x = self.dense_0(torch.cat([start_states, cls_token_state], dim=-1))\n        x = self.activation(x)\n        x = self.dense_1(x).squeeze(-1)\n\n        return x\n\n\n@dataclass\nclass SquadHeadOutput(ModelOutput):\n    """\n    Base class for outputs of question answering models using a [`~modeling_utils.SQuADHead`].\n\n    Args:\n        loss (`torch.FloatTensor` of shape `(1,)`, *optional*, returned if both `start_positions` and `end_positions` are provided):\n            Classification loss as the sum of start token, end token (and is_impossible if provided) classification\n            losses.\n        start_top_log_probs (`torch.FloatTensor` of shape `(batch_size, config.start_n_top)`, *optional*, returned if `start_positions` or `end_positions` is not provided):\n            Log probabilities for the top config.start_n_top start token possibilities (beam-search).\n        start_top_index (`torch.LongTensor` of shape `(batch_size, config.start_n_top)`, *optional*, returned if `start_positions` or `end_positions` is not provided):\n            Indices for the top config.start_n_top start token possibilities (beam-search).\n        end_top_log_probs (`torch.FloatTensor` of shape `(batch_size, config.start_n_top * config.end_n_top)`, *optional*, returned if `start_positions` or `end_positions` is not provided):\n            Log probabilities for the top `config.start_n_top * config.end_n_top` end token possibilities\n            (beam-search).\n        end_top_index (`torch.LongTensor` of shape `(batch_size, config.start_n_top * config.end_n_top)`, *optional*, returned if `start_positions` or `end_positions` is not provided):\n            Indices for the top `config.start_n_top * config.end_n_top` end token possibilities (beam-search).\n        cls_logits (`torch.FloatTensor` of shape `(batch_size,)`, *optional*, returned if `start_positions` or `end_positions` is not provided):\n            Log probabilities for the `is_impossible` label of the answers.\n\n    """\n\n    loss: Optional[torch.FloatTensor] = None\n    start_top_log_probs: Optional[torch.FloatTensor] = None\n    start_top_index: Optional[torch.LongTensor] = None\n    end_top_log_probs: Optional[torch.FloatTensor] = None\n    end_top_index: Optional[torch.LongTensor] = None\n    cls_logits: Optional[torch.FloatTensor] = None\n\n\nclass SQuADHead(nn.Module):\n    r"""\n    A SQuAD head inspired by XLNet.\n\n    Args:\n        config ([`PretrainedConfig`]):\n            The config used by the model, will be used to grab the `hidden_size` of the model and the `layer_norm_eps`\n            to use.\n    """\n\n    def __init__(self, config):\n        super().__init__()\n        self.start_n_top = config.start_n_top\n        self.end_n_top = config.end_n_top\n\n        self.start_logits = PoolerStartLogits(config)\n        self.end_logits = PoolerEndLogits(config)\n        self.answer_class = PoolerAnswerClass(config)\n\n    @replace_return_docstrings(output_type=SquadHeadOutput, config_class=PretrainedConfig)\n    def forward(\n        self,\n        hidden_states: torch.FloatTensor,\n        start_positions: Optional[torch.LongTensor] = None,\n        end_positions: Optional[torch.LongTensor] = None,\n        cls_index: Optional[torch.LongTensor] = None,\n        is_impossible: Optional[torch.LongTensor] = None,\n        p_mask: Optional[torch.FloatTensor] = None,\n        return_dict: bool = False,\n    ) -> Union[SquadHeadOutput, Tuple[torch.FloatTensor]]:\n        """\n        Args:\n            hidden_states (`torch.FloatTensor` of shape `(batch_size, seq_len, hidden_size)`):\n                Final hidden states of the model on the sequence tokens.\n            start_positions (`torch.LongTensor` of shape `(batch_size,)`, *optional*):\n                Positions of the first token for the labeled span.\n            end_positions (`torch.LongTensor` of shape `(batch_size,)`, *optional*):\n                Positions of the last token for the labeled span.\n            cls_index (`torch.LongTensor` of shape `(batch_size,)`, *optional*):\n                Position of the CLS token for each sentence in the batch. If `None`, takes the last token.\n            is_impossible (`torch.LongTensor` of shape `(batch_size,)`, *optional*):\n                Whether the question has a possible answer in the paragraph or not.\n            p_mask (`torch.FloatTensor` of shape `(batch_size, seq_len)`, *optional*):\n                Mask for tokens at invalid position, such as query and special symbols (PAD, SEP, CLS). 1.0 means token\n                should be masked.\n            return_dict (`bool`, *optional*, defaults to `False`):\n                Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.\n\n        Returns:\n        """\n        start_logits = self.start_logits(hidden_states, p_mask=p_mask)\n\n        if start_positions is not None and end_positions is not None:\n            # If we are on multi-GPU, let\'s remove the dimension added by batch splitting\n            for x in (start_positions, end_positions, cls_index, is_impossible):\n                if x is not None and x.dim() > 1:\n                    x.squeeze_(-1)\n\n            # during training, compute the end logits based on the ground truth of the start position\n            end_logits = self.end_logits(hidden_states, start_positions=start_positions, p_mask=p_mask)\n\n            loss_fct = CrossEntropyLoss()\n            start_loss = loss_fct(start_logits, start_positions)\n            end_loss = loss_fct(end_logits, end_positions)\n            total_loss = (start_loss + end_loss) / 2\n\n            if cls_index is not None and is_impossible is not None:\n                # Predict answerability from the representation of CLS and START\n                cls_logits = self.answer_class(hidden_states, start_positions=start_positions, cls_index=cls_index)\n                loss_fct_cls = nn.BCEWithLogitsLoss()\n                cls_loss = loss_fct_cls(cls_logits, is_impossible)\n\n                # note(zhiliny): by default multiply the loss by 0.5 so that the scale is comparable to start_loss and end_loss\n                total_loss += cls_loss * 0.5\n\n            return SquadHeadOutput(loss=total_loss) if return_dict else (total_loss,)\n\n        else:\n            # during inference, compute the end logits based on beam search\n            bsz, slen, hsz = hidden_states.size()\n            start_log_probs = nn.functional.softmax(start_logits, dim=-1)  # shape (bsz, slen)\n\n            start_top_log_probs, start_top_index = torch.topk(\n                start_log_probs, self.start_n_top, dim=-1\n            )  # shape (bsz, start_n_top)\n            start_top_index_exp = start_top_index.unsqueeze(-1).expand(-1, -1, hsz)  # shape (bsz, start_n_top, hsz)\n            start_states = torch.gather(hidden_states, -2, start_top_index_exp)  # shape (bsz, start_n_top, hsz)\n            start_states = start_states.unsqueeze(1).expand(-1, slen, -1, -1)  # shape (bsz, slen, start_n_top, hsz)\n\n            hidden_states_expanded = hidden_states.unsqueeze(2).expand_as(\n                start_states\n            )  # shape (bsz, slen, start_n_top, hsz)\n            p_mask = p_mask.unsqueeze(-1) if p_mask is not None else None\n            end_logits = self.end_logits(hidden_states_expanded, start_states=start_states, p_mask=p_mask)\n            end_log_probs = nn.functional.softmax(end_logits, dim=1)  # shape (bsz, slen, start_n_top)\n\n            end_top_log_probs, end_top_index = torch.topk(\n                end_log_probs, self.end_n_top, dim=1\n            )  # shape (bsz, end_n_top, start_n_top)\n            end_top_log_probs = end_top_log_probs.view(-1, self.start_n_top * self.end_n_top)\n            end_top_index = end_top_index.view(-1, self.start_n_top * self.end_n_top)\n\n            start_states = torch.einsum("blh,bl->bh", hidden_states, start_log_probs)\n            cls_logits = self.answer_class(hidden_states, start_states=start_states, cls_index=cls_index)\n\n            if not return_dict:\n                return (start_top_log_probs, start_top_index, end_top_log_probs, end_top_index, cls_logits)\n            else:\n                return SquadHeadOutput(\n                    start_top_log_probs=start_top_log_probs,\n                    start_top_index=start_top_index,\n                    end_top_log_probs=end_top_log_probs,\n                    end_top_index=end_top_index,\n                    cls_logits=cls_logits,\n                )\n\n\nclass SequenceSummary(nn.Module):\n    r"""\n    Compute a single vector summary of a sequence hidden states.\n\n    Args:\n        config ([`PretrainedConfig`]):\n            The config used by the model. Relevant arguments in the config class of the model are (refer to the actual\n            config class of your model for the default values it uses):\n\n            - **summary_type** (`str`) -- The method to use to make this summary. Accepted values are:\n\n                - `"last"` -- Take the last token hidden state (like XLNet)\n                - `"first"` -- Take the first token hidden state (like Bert)\n                - `"mean"` -- Take the mean of all tokens hidden states\n                - `"cls_index"` -- Supply a Tensor of classification token position (GPT/GPT-2)\n                - `"attn"` -- Not implemented now, use multi-head attention\n\n            - **summary_use_proj** (`bool`) -- Add a projection after the vector extraction.\n            - **summary_proj_to_labels** (`bool`) -- If `True`, the projection outputs to `config.num_labels` classes\n              (otherwise to `config.hidden_size`).\n            - **summary_activation** (`Optional[str]`) -- Set to `"tanh"` to add a tanh activation to the output,\n              another string or `None` will add no activation.\n            - **summary_first_dropout** (`float`) -- Optional dropout probability before the projection and activation.\n            - **summary_last_dropout** (`float`)-- Optional dropout probability after the projection and activation.\n    """\n\n    def __init__(self, config: PretrainedConfig):\n        super().__init__()\n\n        self.summary_type = getattr(config, "summary_type", "last")\n        if self.summary_type == "attn":\n            # We should use a standard multi-head attention module with absolute positional embedding for that.\n            # Cf. https://github.com/zihangdai/xlnet/blob/master/modeling.py#L253-L276\n            # We can probably just use the multi-head attention module of PyTorch >=1.1.0\n            raise NotImplementedError\n\n        self.summary = Identity()\n        if hasattr(config, "summary_use_proj") and config.summary_use_proj:\n            if hasattr(config, "summary_proj_to_labels") and config.summary_proj_to_labels and config.num_labels > 0:\n                num_classes = config.num_labels\n            else:\n                num_classes = config.hidden_size\n            self.summary = nn.Linear(config.hidden_size, num_classes)\n\n        activation_string = getattr(config, "summary_activation", None)\n        self.activation: Callable = get_activation(activation_string) if activation_string else Identity()\n\n        self.first_dropout = Identity()\n        if hasattr(config, "summary_first_dropout") and config.summary_first_dropout > 0:\n            self.first_dropout = nn.Dropout(config.summary_first_dropout)\n\n        self.last_dropout = Identity()\n        if hasattr(config, "summary_last_dropout") and config.summary_last_dropout > 0:\n            self.last_dropout = nn.Dropout(config.summary_last_dropout)\n\n    def forward(\n        self, hidden_states: torch.FloatTensor, cls_index: Optional[torch.LongTensor] = None\n    ) -> torch.FloatTensor:\n        """\n        Compute a single vector summary of a sequence hidden states.\n\n        Args:\n            hidden_states (`torch.FloatTensor` of shape `[batch_size, seq_len, hidden_size]`):\n                The hidden states of the last layer.\n            cls_index (`torch.LongTensor` of shape `[batch_size]` or `[batch_size, ...]` where ... are optional leading dimensions of `hidden_states`, *optional*):\n                Used if `summary_type == "cls_index"` and takes the last token of the sequence as classification token.\n\n        Returns:\n            `torch.FloatTensor`: The summary of the sequence hidden states.\n        """\n        if self.summary_type == "last":\n            output = hidden_states[:, -1]\n        elif self.summary_type == "first":\n            output = hidden_states[:, 0]\n        elif self.summary_type == "mean":\n            output = hidden_states.mean(dim=1)\n        elif self.summary_type == "cls_index":\n            if cls_index is None:\n                cls_index = torch.full_like(\n                    hidden_states[..., :1, :],\n                    hidden_states.shape[-2] - 1,\n                    dtype=torch.long,\n                )\n            else:\n                cls_index = cls_index.unsqueeze(-1).unsqueeze(-1)\n                cls_index = cls_index.expand((-1,) * (cls_index.dim() - 1) + (hidden_states.size(-1),))\n            # shape of cls_index: (bsz, XX, 1, hidden_size) where XX are optional leading dim of hidden_states\n            output = hidden_states.gather(-2, cls_index).squeeze(-2)  # shape (bsz, XX, hidden_size)\n        elif self.summary_type == "attn":\n            raise NotImplementedError\n\n        output = self.first_dropout(output)\n        output = self.summary(output)\n        output = self.activation(output)\n        output = self.last_dropout(output)\n\n        return output\n\n\ndef unwrap_model(model: nn.Module) -> nn.Module:\n    """\n    Recursively unwraps a model from potential containers (as used in distributed training).\n\n    Args:\n        model (`torch.nn.Module`): The model to unwrap.\n    """\n    # since there could be multiple levels of wrapping, unwrap recursively\n    if hasattr(model, "module"):\n        return unwrap_model(model.module)\n    else:\n        return model\n\n\ndef expand_device_map(device_map, param_names, start_prefix):\n    """\n    Expand a device map to return the correspondance parameter name to device.\n    """\n    new_device_map = {}\n    param_names = [p[len(start_prefix) :] for p in param_names if p.startswith(start_prefix)]\n    for module, device in device_map.items():\n        new_device_map.update(\n            {p: device for p in param_names if p == module or p.startswith(f"{module}.") or module == ""}\n        )\n    return new_device_map\n\n\ndef get_disk_only_shard_files(device_map, sharded_metadata, start_prefix):\n    """\n    Returns the list of shard files containing only weights offloaded to disk.\n    """\n\n    weight_map = {\n        p[len(start_prefix) :]: v for p, v in sharded_metadata["weight_map"].items() if p.startswith(start_prefix)\n    }\n    files_content = collections.defaultdict(list)\n    for weight_name, filename in weight_map.items():\n        while len(weight_name) > 0 and weight_name not in device_map:\n            weight_name = ".".join(weight_name.split(".")[:-1])\n        files_content[filename].append(device_map[weight_name])\n\n    return [fname for fname, devices in files_content.items() if set(devices) == {"disk"}]\n')

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors, Facebook AI Research authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import collections
import functools
import gc
import importlib.metadata
import inspect
import json
import os
import re
import shutil
import tempfile
import warnings
from contextlib import contextmanager
from dataclasses import dataclass
from functools import pa